# 3D Droplet Oscillation (Part 2, Postprocessing - Energies)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//wmg.Sessions

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
var workSess = wmg.Sessions.Skip(numSess);
workSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/30/2022 08:14:52	5cdca669...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2*	05/27/2022 20:59:40	87cde3b9...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2*	05/27/2022 20:59:17	33d67916...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2*	05/27/2022 20:58:55	efea163a...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/27/2022 20:58:32	91e4a6b9...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart2*	05/27/2022 20:58:11	7c6f2773...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2*	05/27/2022 20:57:50	a2d5a18a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2*	05/27/2022 20:57:27	99a785fa...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart2*	05/27/2022 20:57:05	7534accc...
#9: OscillatingDrop

In [ ]:
//workSess.Pick(0).Delete(true);

Session 87ef392c-1c5f-43a6-8738-01634830ab3c deleted.


In [ ]:
//var targetDB = databases.Pick(0);
//wmg.Sessions.Skip(numSess).Take(14).CopyAll(targetDB); //Move(targetDB); //MoveAll(targetDB);
// var delSess = workSess.Take(6);
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()
// delSess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015*	05/30/2022 09:49:51	03e7638e...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03*	05/30/2022 09:49:24	2fa067c1...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	05/30/2022 09:48:58	543c23bc...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit*	05/30/2022 09:48:34	2fc7f304...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit*	05/30/2022 09:48:10	78f416af...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit*	05/30/2022 09:47:47	2ff0f605...


In [ ]:
// foreach(var dS in delSess) {
//     dS.Delete(true);
// }

Session 03e7638e-613a-4ae6-a22e-f57820722ff5 deleted.
Session 2fa067c1-ff54-4405-b737-df094e5b0be2 deleted.
Session 543c23bc-96a4-4cc2-a193-4ba9b2e522ba deleted.
Session 2fc7f304-64d1-4ce3-b2f3-99fae7f6c886 deleted.
Session 78f416af-c991-42cc-b791-ab5c6ed559b4 deleted.
Session 2ff0f605-e77f-43b8-854f-081aa8a14c14 deleted.


In [ ]:
var sess = wmg.Sessions.Skip(numSess).Pick(29);
sess

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart1*	05/17/2022 08:52:06	3763b09d...

In [ ]:
sess.GetSessionDirectory()

\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D\sessions\8e7bc526-ef62-478a-b4c8-05a91e24944c

In [ ]:
sess.DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Apr08_113220

In [ ]:
sess.Timesteps

#0:  { Time-step: 417.0; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 417.1; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 417; Physical time: 2.0849999999999778s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#3:  { Time-step: 418; Physical time: 2.0899999999999777s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#4:  { Time-step: 419; Physical time: 2.0949999999999775s; Fields: Phi, PhiDG, VelocityX

In [ ]:
sess.Timesteps.Skip(5).Every(10).Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart1__3763b09d-e0d2-4e1b-ad44-d1675652c728


In [ ]:
using System.IO;

# Study Overview

In [ ]:
var studySess = workSess.Skip(0).Take(18);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/30/2022 08:14:52	5cdca669...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_restart2*	05/27/2022 20:59:40	87cde3b9...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2*	05/27/2022 20:59:17	33d67916...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart2*	05/27/2022 20:58:55	efea163a...
#4: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart2*	05/27/2022 20:58:32	91e4a6b9...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart2*	05/27/2022 20:58:11	7c6f2773...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2*	05/27/2022 20:57:50	a2d5a18a...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2*	05/27/2022 20:57:27	99a785fa...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart2*	05/27/2022 20:57:05	7534accc...
#9: OscillatingDrop

In [ ]:
// string oldName = studySess.Pick(0).Name;
// oldName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart1_restart2

In [ ]:
// string  newName = oldName.Substring(0, oldName.Length - 10) + '2';
// newName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2

In [ ]:
// studySess.Pick(0).Name = newName

OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_restart2

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01", "m3_Oh01_eta04", "m3_Oh01_eta03", "m3_Oh01_eta015", "m4_Oh01_eta04", "m4_Oh01_eta01", "m4_Oh056_eta005"};
string[] gridS = new string[] { "J686", "J432" };
bool[] ShouldAnaInit = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[caseS.Length, 2];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
for(int iCol = 0; iCol < PlotTableEnrgy.GetLength(1); iCol++) {
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
   
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }

    if(SIs.IsNullOrEmpty()) 
    continue;
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }

    var plot = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    //fmt.LineColor = LineColors.Blue;
    if (ShouldAnaInit[iAna]) 
        fmt.DashType = DashTypes.Dashed;
    else  
        fmt.DashType = DashTypes.Solid;   
    var fmtSE = fmt.CloneAs();   
    fmtSE.LineColor = LineColors.Blue;    
    var fmtKE = fmt.CloneAs();   
    fmtKE.LineColor = LineColors.Red;    
    var fmtTE = fmt.CloneAs();   
    fmtTE.LineColor = LineColors.Blue;    

    string description = _grd;
    if(ShouldAnaInit[iAna]) 
        description = description + "-3OrdInit";
    if(useNewton[iNew]) 
        description = description + "-Newton";
    string nameSE = "surfaceEnergy "+description;
    string nameKE = "kineticEnergy "+description;
    string nameTE = "totalEnergy "+description;


    List<double> time = new List<double>();
    List<double> procSurfEnrgy = new List<double>();
    List<double> kinEnrgy = new List<double>();
    List<double> totEnrgy = new List<double>();


    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session: {0}", pSI.Name);
        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }
        
        time = time.Concat(tab["time"]).ToList();
        
        if(iCol == 0) {
            foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
                procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
            }  
            kinEnrgy = kinEnrgy.Concat(tab["kineticEnergy"]).ToList();
        }
    
        if(iCol == 1) {
            totEnrgy = totEnrgy.Concat(tab["totalEnergy"]).ToList(); 
        }

    }

    if(iCol == 0) {
        plot.AddDataGroup(nameSE, time, procSurfEnrgy, fmtSE);
        plot.AddDataGroup(nameKE, time, kinEnrgy, fmtKE);
    }
    if(iCol == 1) {
        plot.AddDataGroup(nameTE, time, totEnrgy, fmtTE);
    }
    
   
    //plot.ShowLegend = iCol == 2 && iRow == 2;
    plot.ShowLegend = true;
    
    //plot.Title = SI.Name;
        
    if(PlotTableEnrgy[iRow,iCol]==null) {
        PlotTableEnrgy[iRow,iCol] = plot;
    } else {
        PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
    }
        
    PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
    PlotTableEnrgy[iRow,iCol].XrangeMax = 7;
}
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart2
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta03
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta015
OD3D_J686k3_w

In [ ]:
PlotTableEnrgy

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 9 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 10 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 11 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 12 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 13 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 14 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 15 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 16 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gn

## Case1 (m2_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.02 
 
 
 
 
 0.04 
 
 
 
 
 0.06 
 
 
 
 
 0.08 
 
 
 
 
 0.1 
 
 
 
 
 0.12 
 
 
 
 
 0.14 
 
 
 
 
 0.16 
 
 
 
 
 0.18 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686 
 
 
 
 
 surfaceEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M62.2,37.9 L62.7,37.9 L63.2,38.1 L63.7,38.5 L64.2,39.0 L64.7,39.8
 L65.1,40.7 L65.6,41.7 L66.1,42.9 L66.6,44.3 L67.1,45.7 L67.6,47.3 L68.1,49.1 L68.6,50.9
 L69.1,52.8 L69.6,54.9 L70.1,57.0 L70.6,59.2 L71.0,61.5 L71.5,63.9 L72.0,66.4 L72.5,69.0
 L73.0,71.6 L73.5,74.3 L74.0,77.1 L74.5,79.9 L75.0,82.8 L75.5,85.8 L76.0,88.8 L76.4,91.9
 L76.9,95.0 L77.4,98.2 L77.9,101.4 L78.4,104.7 L78.9,108.1 L79.4,111.5 L79.9,114.9 L80.4,118.4
 L80.9,121.9 L81.4,125.5 L81.9,129.1 L82.3,132.8 L82.8,136.5 L83.3,140.2 L83.8,144.0 L84.3,147.9
 L84.8,151.7 L85.3,155.6 L85.8,159.6 L86.3,163.5 L86.8,167.5 L87.3,171.6 L87.8,175.6 L88.2,179.7
 L88.7,183.8 L89.2,188.0 L89.7,192.2 L90.2,196.4 L90.7,200.6 L91.2,204.9 L91.7,209.2 L92.2,213.5
 L92.7,217.8 L93.2,222.2 L93.6,226.6 L94.1,231.0 L94.6,235.4 L95.1,239.9 L95.6,244.3 L96.1,248.7
 L96.6,253.3 L97.1,257.8 L97.6,262.3 L98.1,266.9 L98.6,271.4 L99.1,276.0 L99.5,280.6 L100.0,285.2
 L100.5,289.8 L101.0,294.4 L101.5,299.0 L102.0,303.6 L102.5,308.2 L103.0,312.9 L103.5,317.5 L104.0,322.1
 L104.5,326.8 L104.9,331.4 L105.4,336.1 L105.9,340.7 L106.4,345.3 L106.9,350.0 L107.4,354.6 L107.9,359.3
 L108.4,363.9 L108.9,368.5 L109.4,373.1 L109.9,377.7 L110.4,382.3 L110.8,386.9 L111.3,391.5 L111.8,396.0
 L112.3,400.6 L112.8,405.1 L113.3,409.6 L113.8,414.1 L114.3,418.5 L114.8,423.0 L115.3,427.4 L115.8,431.8
 L116.2,436.1 L116.7,440.4 L117.2,444.7 L117.7,449.0 L118.2,453.2 L118.7,457.4 L119.2,461.5 L119.7,465.6
 L120.2,469.7 L120.7,473.7 L121.2,477.6 L121.7,481.5 L122.1,485.3 L122.6,489.1 L123.1,492.8 L123.6,496.4
 L124.1,500.0 L124.6,503.4 L125.1,506.9 L125.6,510.2 L126.1,513.4 L126.6,516.6 L127.1,519.6 L127.6,522.7
 L128.0,525.6 L128.5,528.4 L129.0,531.1 L129.5,533.7 L130.0,536.2 L130.5,538.5 L131.0,540.8 L131.5,542.9
 L132.0,545.1 L132.5,547.0 L133.0,548.9 L133.4,550.6 L133.9,552.1 L134.4,553.6 L134.9,554.9 L135.4,556.2
 L135.9,557.4 L136.4,558.4 L136.9,559.2 L137.4,560.0 L137.9,560.6 L138.4,561.2 L138.9,561.6 L139.3,561.9
 L139.8,562.0 L140.3,562.1 L140.8,562.0 L141.3,561.8 L141.8,561.5 L142.3,561.1 L142.8,560.6 L143.3,560.0
 L143.8,559.2 L144.3,558.4 L144.7,557.4 L145.2,556.4 L145.7,555.3 L146.2,554.0 L146.7,552.7 L147.2,551.2
 L147.7,549.7 L148.2,548.0 L148.7,546.2 L149.2,544.5 L149.7,542.6 L150.2,540.6 L150.6,538.6 L151.1,536.5
 L151.6,534.3 L152.1,532.0 L152.6,529.7 L153.1,527.3 L153.6,524.8 L154.1,522.2 L154.6,519.7 L155.1,517.0
 L155.6,514.5 L156.0,511.8 L156.5,509.1 L157.0,506.3 L157.5,503.5 L158.0,500.5 L158.5,497.7 L159.0,494.8
 L159.5,491.8 L160.0,488.8 L160.5,485.7 L161.0,482.8 L161.5,479.7 L161.9,476.7 L162.4,473.7 L162.9,470.7
 L163.4,467.7 L163.9,464.7 L164.4,461.8 L164.9,458.8 L165.4,455.9 L165.9,453.0 L166.4,450.1 L166.9,447.2
 L167.4,444.4 L167.8,441.6 L168.3,438.8 L168.8,436.1 L169.3,433.4 L169.8,430.8 L170.3,428.2 L170.8,425.6
 L171.3,423.2 L171.8,420.8 L172.3,418.4 L172.8,416.1 L173.2,413.8 L173.7,411.6 L174.2,409.5 L174.7,407.4
 L175.2,405.4 L175.7,403.5 L176.2,401.6 L176.7,399.8 L177.2,398.1 L177.7,396.5 L178.2,394.9 L178.7,393.5
 L179.1,392.1 L179.6,390.8 L180.1,389.6 L180.6,388.4 L181.1,387.4 L181.6,386.4 L182.1,385.5 L182.6,384.8
 L183.1,384.1 L183.6,383.4 L184.1,382.9 L184.5,382.4 L185.0,382.

In [ ]:
var gp = PlotTableEnrgy[0,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.16 
 
 
 
 
 3.18 
 
 
 
 
 3.2 
 
 
 
 
 3.22 
 
 
 
 
 3.24 
 
 
 
 
 3.26 
 
 
 
 
 3.28 
 
 
 
 
 3.3 
 
 
 
 
 3.32 
 
 
 
 
 3.34 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686 
 
 
 
 
 totalEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M510.8,57.1 L564.2,57.1 M62.2,86.3 L62.7,86.2 L63.2,86.1 L63.7,86.1 L64.2,86.0 L64.7,86.0
 L65.1,86.1 L65.6,86.1 L66.1,86.2 L66.6,86.3 L67.1,86.4 L67.6,86.6 L68.1,86.8 L68.6,87.0
 L69.1,87.3 L69.6,87.5 L70.1,87.8 L70.6,88.1 L71.0,88.5 L71.5,88.8 L72.0,89.2 L72.5,89.6
 L73.0,90.0 L73.5,90.5 L74.0,91.0 L74.5,91.4 L75.0,91.9 L75.5,92.4 L76.0,93.0 L76.4,93.5
 L76.9,94.1 L77.4,94.6 L77.9,95.2 L78.4,95.8 L78.9,96.5 L79.4,97.1 L79.9,97.8 L80.4,98.4
 L80.9,99.1 L81.4,99.8 L81.9,100.5 L82.3,101.2 L82.8,102.0 L83.3,102.7 L83.8,103.5 L84.3,104.3
 L84.8,105.1 L85.3,105.9 L85.8,106.7 L86.3,107.5 L86.8,108.4 L87.3,109.2 L87.8,110.1 L88.2,111.0
 L88.7,111.9 L89.2,112.8 L89.7,113.7 L90.2,114.7 L90.7,115.6 L91.2,116.6 L91.7,117.6 L92.2,118.6
 L92.7,119.6 L93.2,120.6 L93.6,121.7 L94.1,122.7 L94.6,123.8 L95.1,124.9 L95.6,126.0 L96.1,127.0
 L96.6,128.2 L97.1,129.3 L97.6,130.5 L98.1,131.7 L98.6,132.9 L99.1,134.1 L99.5,135.3 L100.0,136.6
 L100.5,137.8 L101.0,139.1 L101.5,140.3 L102.0,141.6 L102.5,143.0 L103.0,144.3 L103.5,145.6 L104.0,147.0
 L104.5,148.4 L104.9,149.7 L105.4,151.1 L105.9,152.6 L106.4,154.0 L106.9,155.5 L107.4,157.0 L107.9,158.5
 L108.4,160.0 L108.9,161.6 L109.4,163.1 L109.9,164.7 L110.4,166.3 L110.8,167.9 L111.3,169.6 L111.8,171.3
 L112.3,173.0 L112.8,174.7 L113.3,176.5 L113.8,178.3 L114.3,180.1 L114.8,181.9 L115.3,183.8 L115.8,185.6
 L116.2,187.5 L116.7,189.5 L117.2,191.4 L117.7,193.4 L118.2,195.4 L118.7,197.5 L119.2,199.6 L119.7,201.7
 L120.2,203.8 L120.7,206.0 L121.2,208.2 L121.7,210.4 L122.1,212.6 L122.6,214.9 L123.1,217.2 L123.6,219.5
 L124.1,221.8 L124.6,224.1 L125.1,226.5 L125.6,229.0 L126.1,231.3 L126.6,233.8 L127.1,236.2 L127.6,238.7
 L128.0,241.2 L128.5,243.7 L129.0,246.2 L129.5,248.7 L130.0,251.2 L130.5,253.7 L131.0,256.2 L131.5,258.8
 L132.0,261.4 L132.5,263.9 L133.0,266.5 L133.4,269.0 L133.9,271.4 L134.4,274.0 L134.9,276.5 L135.4,279.0
 L135.9,281.6 L136.4,284.0 L136.9,286.5 L137.4,289.0 L137.9,291.4 L138.4,293.9 L138.9,296.3 L139.3,298.7
 L139.8,301.1 L140.3,303.5 L140.8,305.8 L141.3,308.2 L141.8,310.5 L142.3,312.7 L142.8,315.0 L143.3,317.3
 L143.8,319.4 L144.3,321.6 L144.7,323.7 L145.2,325.9 L145.7,328.0 L146.2,330.0 L146.7,332.0 L147.2,334.0
 L147.7,336.0 L148.2,337.9 L148.7,339.7 L149.2,341.6 L149.7,343.4 L150.2,345.2 L150.6,347.0 L151.1,348.7
 L151.6,350.3 L152.1,352.0 L152.6,353.6 L153.1,355.1 L153.6,356.6 L154.1,358.0 L154.6,359.5 L155.1,360.8
 L155.6,362.3 L156.0,363.6 L156.5,364.9 L157.0,366.2 L157.5,367.4 L158.0,368.4 L158.5,369.7 L159.0,370.8
 L159.5,371.8 L160.0,372.8 L160.5,373.7 L161.0,374.7 L161.5,375.5 L161.9,376.4 L162.4,377.2 L162.9,378.1
 L163.4,378.9 L163.9,379.6 L164.4,380.3 L164.9,381.0 L165.4,381.7 L165.9,382.3 L166.4,383.0 L166.9,383.6
 L167.4,384.1 L167.8,384.7 L168.3,385.2 L168.8,385.7 L169.3,386.1 L169.8,386.6 L170.3,387.1 L170.8,387.4
 L171.3,387.8 L171.8,388.2 L172.3,388.5 L172.8,388.8 L173.2,389.1 L173.7,389.4 L174.2,389.7 L174.7,389.9
 L175.2,390.2 L175.7,390.4 L176.2,390.6 L176.7,390.8 L177.2,390.9 L177.7,391.1 L178.2,391.3 L178.7,391.4
 L179.1,391.5 L179.6,391.6 L180.1,391.8 L180.6,391.9 L181.1,392.0 L181.6,392.0 L182.1,392.1 L182.6,392.2
 L183.1,392.3 L183.6,392.3 L184.1,392.4 L184.5,39

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[0,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m2/" + caseS[0] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 2 (m2_Oh01_eta02)

In [ ]:
var gp = PlotTableEnrgy[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0.04 
 
 
 
 
 0.045 
 
 
 
 
 0.05 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M70.5,53.7 L71.0,53.5 L71.5,53.5 L72.0,53.6 L72.4,53.8 L72.9,54.1
 L73.4,54.7 L73.9,55.3 L74.4,56.1 L74.9,57.1 L75.4,58.2 L75.8,59.5 L76.3,60.9 L76.8,62.5
 L77.3,64.1 L77.8,66.0 L78.3,68.0 L78.8,70.1 L79.2,72.4 L79.7,74.8 L80.2,77.3 L80.7,79.9
 L81.2,82.6 L81.7,85.5 L82.2,88.5 L82.6,91.5 L83.1,94.7 L83.6,98.0 L84.1,101.3 L84.6,104.8
 L85.1,108.3 L85.5,112.0 L86.0,115.7 L86.5,119.5 L87.0,123.3 L87.5,127.3 L88.0,131.3 L88.5,135.4
 L88.9,139.6 L89.4,143.9 L89.9,148.0 L90.4,152.5 L90.9,157.0 L91.4,161.5 L91.9,166.1 L92.3,170.8
 L92.8,175.5 L93.3,180.1 L93.8,185.0 L94.3,189.9 L94.8,194.9 L95.3,199.7 L95.7,204.9 L96.2,210.0
 L96.7,215.1 L97.2,220.3 L97.7,225.4 L98.2,230.8 L98.7,236.1 L99.1,241.5 L99.6,246.9 L100.1,252.2
 L100.6,257.7 L101.1,263.2 L101.6,268.6 L102.1,274.2 L102.5,279.8 L103.0,285.4 L103.5,291.0 L104.0,296.6
 L104.5,302.2 L105.0,307.8 L105.5,313.5 L105.9,319.1 L106.4,324.7 L106.9,330.4 L107.4,336.0 L107.9,341.6
 L108.4,347.2 L108.8,352.8 L109.3,358.4 L109.8,363.9 L110.3,369.3 L110.8,374.9 L111.3,380.2 L111.8,385.8
 L112.2,391.2 L112.7,396.4 L113.2,401.8 L113.7,407.1 L114.2,412.2 L114.7,417.5 L115.2,422.6 L115.6,427.7
 L116.1,432.7 L116.6,437.7 L117.1,442.6 L117.6,447.4 L118.1,452.1 L118.6,456.6 L119.0,461.1 L119.5,465.8
 L120.0,470.2 L120.5,474.6 L121.0,478.9 L121.5,482.9 L122.0,487.1 L122.4,491.1 L122.9,495.0 L123.4,498.7
 L123.9,502.5 L124.4,506.2 L124.9,509.7 L125.4,513.0 L125.8,516.4 L126.3,519.6 L126.8,522.7 L127.3,525.7
 L127.8,528.6 L128.3,531.3 L128.8,534.0 L129.2,536.3 L129.7,538.9 L130.2,541.0 L130.7,543.3 L131.2,545.4
 L131.7,547.3 L132.1,549.1 L132.6,550.8 L133.1,552.3 L133.6,553.7 L134.1,555.0 L134.6,556.2 L135.1,557.3
 L135.5,558.2 L136.0,559.0 L136.5,559.7 L137.0,560.3 L137.5,560.8 L138.0,561.1 L138.5,561.3 L138.9,561.5
 L139.4,561.5 L139.9,561.3 L140.4,561.1 L140.9,560.8 L141.4,560.3 L141.9,559.7 L142.3,559.1 L142.8,558.3
 L143.3,557.4 L143.8,556.4 L144.3,555.4 L144.8,554.2 L145.3,552.9 L145.7,551.4 L146.2,549.9 L146.7,548.3
 L147.2,546.8 L147.7,545.1 L148.2,543.3 L148.7,541.4 L149.1,539.4 L149.6,537.3 L150.1,535.2 L150.6,533.0
 L151.1,530.7 L151.6,528.4 L152.1,526.0 L152.5,523.5 L153.0,521.0 L153.5,518.4 L154.0,515.8 L154.5,513.1
 L155.0,510.1 L155.5,507.4 L155.9,504.6 L156.4,501.8 L156.9,498.9 L157.4,496.0 L157.9,493.0 L158.4,490.0
 L158.8,487.0 L159.3,484.0 L159.8,481.0 L160.3,478.0 L160.8,474.9 L161.3,471.8 L161.8,468.8 L162.2,465.7
 L162.7,462.7 L163.2,459.6 L163.7,456.6 L164.2,453.5 L164.7,450.5 L165.2,447.5 L165.6,444.5 L166.1,441.6
 L166.6,438.6 L167.1,435.7 L167.6,432.9 L168.1,430.0 L168.6,427.2 L169.0,424.5 L169.5,421.8 L170.0,419.1
 L170.5,416.5 L171.0,413.9 L171.5,411.4 L172.0,408.9 L172.4,406.5 L172.9,404.2 L173.4,401.5 L173.9,399.4
 L174.4,397.3 L174.9,395.0 L175.4,393.1 L175.8,391.2 L176.3,389.4 L176.8,387.6 L177.3,385.9 L177.8,384.3
 L178.3,382.7 L178.8,381.3 L179.2,379.9 L179.7,378.5 L180.2,377.3 L180.7,376.2 L181.2,375.1 L181.7,374.1
 L182.1,373.2 L182.6,372.4 L183.1,371.7 L183.6,371.1 L184.1,370.5 L184.6,370.0 L185.1,369.7 L185.5,369.4
 L186.0,369.2 L186.5,369.1 L187.0,369.0 L187.5,369.1 L188.0,369.2 L188.5

In [ ]:
var gp = PlotTableEnrgy[1,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.145 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 3.18 
 
 
 
 
 3.185 
 
 
 
 
 3.19 
 
 
 
 
 3.195 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M70.5,74.6 L71.0,75.6 L71.5,76.5 L72.0,77.1 L72.4,77.7 L72.9,78.2
 L73.4,78.6 L73.9,79.0 L74.4,79.3 L74.9,79.6 L75.4,79.9 L75.8,80.3 L76.3,80.6 L76.8,80.9
 L77.3,81.1 L77.8,81.4 L78.3,81.8 L78.8,82.1 L79.2,82.5 L79.7,82.8 L80.2,83.2 L80.7,83.6
 L81.2,84.0 L81.7,84.4 L82.2,84.9 L82.6,85.4 L83.1,85.8 L83.6,86.3 L84.1,86.9 L84.6,87.4
 L85.1,87.9 L85.5,88.5 L86.0,89.1 L86.5,89.7 L87.0,90.3 L87.5,90.9 L88.0,91.6 L88.5,92.3
 L88.9,92.9 L89.4,93.6 L89.9,94.2 L90.4,95.0 L90.9,95.8 L91.4,96.6 L91.9,97.4 L92.3,98.2
 L92.8,99.0 L93.3,99.7 L93.8,100.7 L94.3,101.6 L94.8,102.5 L95.3,103.3 L95.7,104.3 L96.2,105.3
 L96.7,106.3 L97.2,107.3 L97.7,108.2 L98.2,109.4 L98.7,110.5 L99.1,111.6 L99.6,112.7 L100.1,113.8
 L100.6,115.0 L101.1,116.3 L101.6,117.4 L102.1,118.7 L102.5,120.0 L103.0,121.4 L103.5,122.7 L104.0,124.1
 L104.5,125.5 L105.0,127.0 L105.5,128.4 L105.9,129.9 L106.4,131.4 L106.9,133.0 L107.4,134.6 L107.9,136.2
 L108.4,137.8 L108.8,139.5 L109.3,141.2 L109.8,142.9 L110.3,144.5 L110.8,146.4 L111.3,148.1 L111.8,150.0
 L112.2,151.9 L112.7,153.7 L113.2,155.7 L113.7,157.7 L114.2,159.5 L114.7,161.6 L115.2,163.7 L115.6,165.7
 L116.1,167.8 L116.6,169.9 L117.1,172.0 L117.6,174.2 L118.1,176.3 L118.6,178.3 L119.0,180.5 L119.5,182.8
 L120.0,185.1 L120.5,187.4 L121.0,189.6 L121.5,191.8 L122.0,194.2 L122.4,196.5 L122.9,198.9 L123.4,201.1
 L123.9,203.6 L124.4,206.0 L124.9,208.4 L125.4,210.6 L125.8,213.1 L126.3,215.6 L126.8,218.0 L127.3,220.4
 L127.8,222.8 L128.3,225.3 L128.8,227.7 L129.2,230.0 L129.7,232.5 L130.2,234.9 L130.7,237.4 L131.2,239.8
 L131.7,242.2 L132.1,244.6 L132.6,247.0 L133.1,249.4 L133.6,251.8 L134.1,254.2 L134.6,256.5 L135.1,258.9
 L135.5,261.2 L136.0,263.5 L136.5,265.8 L137.0,268.1 L137.5,270.4 L138.0,272.7 L138.5,274.9 L138.9,277.2
 L139.4,279.4 L139.9,281.6 L140.4,283.8 L140.9,285.9 L141.4,288.1 L141.9,290.2 L142.3,292.3 L142.8,294.4
 L143.3,296.4 L143.8,298.4 L144.3,300.4 L144.8,302.4 L145.3,304.4 L145.7,306.1 L146.2,308.0 L146.7,309.9
 L147.2,311.8 L147.7,313.7 L148.2,315.4 L148.7,317.2 L149.1,318.9 L149.6,320.6 L150.1,322.3 L150.6,323.9
 L151.1,325.5 L151.6,327.0 L152.1,328.6 L152.5,330.1 L153.0,331.5 L153.5,333.0 L154.0,334.4 L154.5,335.7
 L155.0,336.9 L155.5,338.2 L155.9,339.5 L156.4,340.8 L156.9,342.0 L157.4,343.1 L157.9,344.3 L158.4,345.4
 L158.8,346.4 L159.3,347.5 L159.8,348.5 L160.3,349.4 L160.8,350.4 L161.3,351.3 L161.8,352.2 L162.2,353.0
 L162.7,353.8 L163.2,354.6 L163.7,355.4 L164.2,356.1 L164.7,356.8 L165.2,357.5 L165.6,358.2 L166.1,358.8
 L166.6,359.4 L167.1,359.9 L167.6,360.5 L168.1,361.0 L168.6,361.5 L169.0,361.9 L169.5,362.4 L170.0,362.8
 L170.5,363.2 L171.0,363.6 L171.5,363.9 L172.0,364.2 L172.4,364.6 L172.9,364.9 L173.4,364.7 L173.9,365.1
 L174.4,365.4 L174.9,365.5 L175.4,365.8 L175.8,366.0 L176.3,366.2 L176.8,366.4 L177.3,366.6 L177.8,366.8
 L178.3,366.9 L178.8,367.1 L179.2,367.2 L179.7,367.3 L180.2,367.4 L180.7,367.5 L181.2,367.6 L181.7,367.7
 L182.1,367.8 L182.6,367.8 L183.1,367.9 L183.6,368.0 L184.1,368.0 L184.6,368.1 L185.1,368.2 L185.5,368.2
 L186.0,368.3 L186.5,368.4 L187.0,368.4 L187.5,368.5 L188.0,368.6 L

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[1,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m2/" + caseS[1] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 3 (m2_Oh01_eta01)

In [ ]:
var gp = PlotTableEnrgy[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.002 
 
 
 
 
 0.004 
 
 
 
 
 0.006 
 
 
 
 
 0.008 
 
 
 
 
 0.01 
 
 
 
 
 0.012 
 
 
 
 
 0.014 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M70.5,96.5 L71.0,96.4 L71.5,96.6 L72.0,96.9 L72.4,97.4 L72.9,98.0
 L73.4,98.8 L73.9,99.7 L74.4,100.8 L74.9,102.0 L75.4,103.2 L75.8,104.8 L76.3,106.5 L76.8,108.4
 L77.3,110.3 L77.8,112.4 L78.3,114.7 L78.8,117.0 L79.2,119.5 L79.7,122.2 L80.2,124.9 L80.7,127.8
 L81.2,130.8 L81.7,133.9 L82.2,137.2 L82.6,140.5 L83.1,144.0 L83.6,147.6 L84.1,151.3 L84.6,155.1
 L85.1,159.0 L85.5,163.0 L86.0,167.1 L86.5,171.3 L87.0,175.6 L87.5,179.9 L88.0,184.4 L88.5,188.9
 L88.9,193.5 L89.4,198.2 L89.9,203.0 L90.4,208.0 L90.9,213.0 L91.4,217.9 L91.9,222.9 L92.3,228.0
 L92.8,232.9 L93.3,238.2 L93.8,243.5 L94.3,248.8 L94.8,254.1 L95.3,259.4 L95.7,264.8 L96.2,270.0
 L96.7,275.6 L97.2,281.1 L97.7,286.6 L98.2,292.1 L98.7,297.4 L99.1,303.1 L99.6,308.7 L100.1,314.2
 L100.6,319.8 L101.1,325.2 L101.6,330.9 L102.1,336.5 L102.5,342.0 L103.0,347.5 L103.5,352.8 L104.0,358.5
 L104.5,364.0 L105.0,369.5 L105.5,374.9 L105.9,380.3 L106.4,385.7 L106.9,391.0 L107.4,396.3 L107.9,401.5
 L108.4,406.7 L108.8,411.9 L109.3,417.0 L109.8,422.0 L110.3,426.7 L110.8,431.8 L111.3,436.8 L111.8,441.6
 L112.2,446.3 L112.7,451.0 L113.2,455.6 L113.7,460.1 L114.2,464.5 L114.7,468.6 L115.2,473.2 L115.6,477.4
 L116.1,481.5 L116.6,485.2 L117.1,489.4 L117.6,493.3 L118.1,497.0 L118.6,500.7 L119.0,504.2 L119.5,507.6
 L120.0,511.0 L120.5,514.2 L121.0,517.4 L121.5,520.4 L122.0,523.3 L122.4,526.1 L122.9,528.8 L123.4,531.4
 L123.9,533.9 L124.4,536.3 L124.9,538.5 L125.4,540.7 L125.8,542.7 L126.3,544.6 L126.8,546.4 L127.3,548.1
 L127.8,549.7 L128.3,551.1 L128.8,552.5 L129.2,553.7 L129.7,554.8 L130.2,555.8 L130.7,556.7 L131.2,557.5
 L131.7,558.1 L132.1,558.7 L132.6,559.1 L133.1,559.5 L133.6,559.7 L134.1,559.8 L134.6,559.8 L135.1,559.7
 L135.5,559.5 L136.0,559.1 L136.5,558.7 L137.0,558.2 L137.5,557.6 L138.0,556.9 L138.5,556.1 L138.9,555.1
 L139.4,554.1 L139.9,553.0 L140.4,551.9 L140.9,550.6 L141.4,549.2 L141.9,547.8 L142.3,546.3 L142.8,544.6
 L143.3,542.6 L143.8,541.0 L144.3,539.3 L144.8,537.4 L145.3,535.5 L145.7,533.5 L146.2,531.4 L146.7,529.2
 L147.2,527.0 L147.7,524.1 L148.2,521.8 L148.7,519.6 L149.1,517.3 L149.6,514.3 L150.1,512.1 L150.6,509.6
 L151.1,507.1 L151.6,504.5 L152.1,501.9 L152.5,499.2 L153.0,496.5 L153.5,493.8 L154.0,491.0 L154.5,488.3
 L155.0,485.5 L155.5,482.7 L155.9,479.9 L156.4,477.1 L156.9,474.2 L157.4,471.4 L157.9,468.6 L158.4,465.8
 L158.8,463.0 L159.3,460.2 L159.8,457.4 L160.3,454.6 L160.8,451.8 L161.3,449.1 L161.8,446.4 L162.2,443.7
 L162.7,441.0 L163.2,438.4 L163.7,435.8 L164.2,433.2 L164.7,430.7 L165.2,428.2 L165.6,425.7 L166.1,423.3
 L166.6,420.9 L167.1,418.6 L167.6,416.3 L168.1,414.1 L168.6,411.9 L169.0,409.8 L169.5,407.8 L170.0,405.8
 L170.5,403.8 L171.0,401.9 L171.5,400.1 L172.0,398.3 L172.4,396.7 L172.9,395.0 L173.4,393.5 L173.9,392.0
 L174.4,390.5 L174.9,389.2 L175.4,387.9 L175.8,386.7 L176.3,384.9 L176.8,384.0 L177.3,383.1 L177.8,382.3
 L178.3,381.5 L178.8,380.7 L179.2,380.1 L179.7,379.5 L180.2,379.0 L180.7,378.5 L181.2,378.1 L181.7,377.9
 L182.1,377.6 L182.6,377.5 L183.1,377.4 L183.6,377.5 L184.1,377.6 L184.6,377.7 L185.1,378.0 L185.5,378.3
 L186.0,378.7 L186.5,379.1 L187.0,379.7 L187.5,380.3 L188.0,381.0 L188.5,381.7 L188.9,382.6 L189.4,383.5
 L189.9,384.4 L190.4,

In [ ]:
var gp = PlotTableEnrgy[2,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.142 
 
 
 
 
 3.144 
 
 
 
 
 3.146 
 
 
 
 
 3.148 
 
 
 
 
 3.15 
 
 
 
 
 3.152 
 
 
 
 
 3.154 
 
 
 
 
 3.156 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M70.5,97.0 L71.0,97.1 L71.5,97.2 L72.0,97.3 L72.4,97.4 L72.9,97.6
 L73.4,97.7 L73.9,97.8 L74.4,98.0 L74.9,98.1 L75.4,98.1 L75.8,98.4 L76.3,98.6 L76.8,98.8
 L77.3,99.0 L77.8,99.3 L78.3,99.5 L78.8,99.8 L79.2,100.0 L79.7,100.3 L80.2,100.6 L80.7,101.0
 L81.2,101.3 L81.7,101.7 L82.2,102.1 L82.6,102.5 L83.1,102.9 L83.6,103.4 L84.1,103.8 L84.6,104.3
 L85.1,104.9 L85.5,105.4 L86.0,106.0 L86.5,106.5 L87.0,107.2 L87.5,107.8 L88.0,108.5 L88.5,109.1
 L88.9,109.9 L89.4,110.6 L89.9,111.4 L90.4,112.3 L90.9,113.2 L91.4,114.0 L91.9,114.9 L92.3,115.8
 L92.8,116.5 L93.3,117.5 L93.8,118.5 L94.3,119.5 L94.8,120.5 L95.3,121.6 L95.7,122.6 L96.2,123.6
 L96.7,124.8 L97.2,126.0 L97.7,127.2 L98.2,128.4 L98.7,129.4 L99.1,130.8 L99.6,132.1 L100.1,133.5
 L100.6,134.8 L101.1,136.0 L101.6,137.5 L102.1,139.0 L102.5,140.4 L103.0,141.9 L103.5,143.2 L104.0,144.9
 L104.5,146.4 L105.0,148.0 L105.5,149.6 L105.9,151.2 L106.4,152.9 L106.9,154.6 L107.4,156.3 L107.9,158.0
 L108.4,159.7 L108.8,161.5 L109.3,163.2 L109.8,165.0 L110.3,166.6 L110.8,168.6 L111.3,170.5 L111.8,172.4
 L112.2,174.3 L112.7,176.2 L113.2,178.1 L113.7,180.1 L114.2,182.0 L114.7,183.7 L115.2,185.9 L115.6,188.0
 L116.1,190.0 L116.6,191.7 L117.1,193.9 L117.6,196.0 L118.1,198.1 L118.6,200.1 L119.0,202.2 L119.5,204.3
 L120.0,206.3 L120.5,208.4 L121.0,210.5 L121.5,212.6 L122.0,214.7 L122.4,216.8 L122.9,218.8 L123.4,220.9
 L123.9,223.0 L124.4,225.1 L124.9,227.2 L125.4,229.3 L125.8,231.4 L126.3,233.4 L126.8,235.5 L127.3,237.6
 L127.8,239.6 L128.3,241.7 L128.8,243.7 L129.2,245.8 L129.7,247.8 L130.2,249.8 L130.7,251.8 L131.2,253.8
 L131.7,255.8 L132.1,257.8 L132.6,259.8 L133.1,261.7 L133.6,263.6 L134.1,265.5 L134.6,267.4 L135.1,269.3
 L135.5,271.2 L136.0,273.1 L136.5,274.9 L137.0,276.7 L137.5,278.5 L138.0,280.3 L138.5,282.0 L138.9,283.8
 L139.4,285.5 L139.9,287.2 L140.4,288.9 L140.9,290.5 L141.4,292.1 L141.9,293.7 L142.3,295.3 L142.8,296.9
 L143.3,298.0 L143.8,299.7 L144.3,301.3 L144.8,302.8 L145.3,304.2 L145.7,305.6 L146.2,307.0 L146.7,308.4
 L147.2,309.7 L147.7,310.4 L148.2,311.7 L148.7,313.2 L149.1,314.5 L149.6,315.2 L150.1,316.6 L150.6,317.8
 L151.1,319.0 L151.6,320.1 L152.1,321.2 L152.5,322.2 L153.0,323.2 L153.5,324.2 L154.0,325.1 L154.5,326.1
 L155.0,326.9 L155.5,327.8 L155.9,328.6 L156.4,329.5 L156.9,330.2 L157.4,331.0 L157.9,331.7 L158.4,332.4
 L158.8,333.1 L159.3,333.8 L159.8,334.4 L160.3,335.0 L160.8,335.6 L161.3,336.2 L161.8,336.7 L162.2,337.2
 L162.7,337.7 L163.2,338.2 L163.7,338.6 L164.2,339.1 L164.7,339.5 L165.2,339.9 L165.6,340.2 L166.1,340.6
 L166.6,340.9 L167.1,341.3 L167.6,341.6 L168.1,341.9 L168.6,342.1 L169.0,342.4 L169.5,342.6 L170.0,342.8
 L170.5,343.1 L171.0,343.3 L171.5,343.4 L172.0,343.6 L172.4,343.8 L172.9,343.9 L173.4,344.1 L173.9,344.2
 L174.4,344.3 L174.9,344.4 L175.4,344.5 L175.8,344.6 L176.3,344.2 L176.8,344.4 L177.3,344.6 L177.8,344.7
 L178.3,344.8 L178.8,344.9 L179.2,345.0 L179.7,345.0 L180.2,345.1 L180.7,345.2 L181.2,345.3 L181.7,345.3
 L182.1,345.4 L182.6,345.4 L183.1,345.5 L183.6,345.6 L184.1,345.6 L184.6,345.7 L185.1,345.8 L185.5,345.9
 L186.0,346.0 L186.5,346.0 L187.0,346.1 L187.5,346.2 L188.0,346.3 L188.5,346.4 L188.9,346.6 L189.4,346.7
 L189.

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[2,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m2/" + caseS[2] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 4 (m3_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 
 kineticEnergyJ686-3OrdInit 
 
 
 kineticEnergyJ686-3OrdInit 
 
 
 
 
 
 surfaceEnergyJ686 
 
 
 surfaceEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.7,105.1 L442.1,105.1 M45.6,549.9 L46.1,549.9 L46.6,550.0 L47.1,550.0 L47.6,550.1 L48.1,550.2
 L48.6,550.3 L49.1,550.4 L49.6,550.5 L50.1,550.7 L50.6,550.8 L51.1,551.0 L51.6,551.2 L52.1,551.4
 L52.6,551.6 L53.1,551.8 L53.7,552.0 L54.2,552.3 L54.7,552.5 L55.2,552.8 L55.7,553.0 L56.2,553.3
 L56.7,553.5 L57.2,553.8 L57.7,554.0 L58.2,554.3 L58.7,554.6 L59.2,554.9 L59.7,555.1 L60.2,555.4
 L60.7,555.7 L61.2,556.0 L61.7,556.2 L62.2,556.5 L62.7,556.8 L63.2,557.1 L63.7,557.3 L64.2,557.6
 L64.7,557.9 L65.2,558.1 L65.7,558.4 L66.2,558.7 L66.7,558.9 L67.2,559.1 L67.7,559.4 L68.2,559.6
 L68.7,559.9 L69.3,560.1 L69.8,560.3 L70.3,560.5 L70.8,560.7 L71.3,560.9 L71.8,561.1 L72.3,561.3
 L72.8,561.5 L73.3,561.6 L73.8,561.8 L74.3,562.0 L74.8,562.1 L75.3,562.2 L75.8,562.4 L76.3,562.5
 L76.8,562.6 L77.3,562.7 L77.8,562.8 L78.3,562.9 L78.8,563.0 L79.3,563.1 L79.8,563.2 L80.3,563.2
 L80.8,563.3 L81.3,563.3 L81.8,563.4 L82.3,563.4 L82.8,563.5 L83.3,563.5 L83.8,563.5 L84.3,563.5
 L84.9,563.5 L85.4,563.5 L85.9,563.5 L86.4,563.5 L86.9,563.5 L87.4,563.4 L87.9,563.4 L88.4,563.4
 L88.9,563.3 L89.4,563.3 L89.9,563.2 L90.4,563.2 L90.9,563.1 L91.4,563.0 L91.9,563.0 L92.4,562.9
 L92.9,562.8 L93.4,562.7 L93.9,562.7 L94.4,562.6 L94.9,562.5 L95.4,562.4 L95.9,562.3 L96.4,562.2
 L96.9,562.1 L97.4,562.1 L97.9,562.0 L98.4,561.9 L98.9,561.8 L99.4,561.7 L99.9,561.6 L100.5,561.5
 L101.0,561.4 L101.5,561.3 L102.0,561.3 L102.5,561.2 L103.0,561.1 L103.5,561.0 L104.0,561.0 L104.5,560.9
 L105.0,560.8 L105.5,560.8 L106.0,560.7 L106.5,560.7 L107.0,560.6 L107.5,560.6 L108.0,560.5 L108.5,560.5
 L109.0,560.5 L109.5,560.4 L110.0,560.4 L110.5,560.4 L111.0,560.4 L111.5,560.4 L112.0,560.4 L112.5,560.4
 L113.0,560.4 L113.5,560.4 L114.0,560.4 L114.5,560.4 L115.0,560.5 L115.5,560.5 L116.1,560.5 L116.6,560.6
 L117.1,560.6 L117.6,560.6 L118.1,560.7 L118.6,560.7 L119.1,560.8 L119.6,560.8 L120.1,560.9 L120.6,561.0
 L121.1,561.0 L121.6,561.1 L122.1,561.1 L122.6,561.2 L123.1,561.3 L123.6,561.3 L124.1,561.4 L124.6,561.5
 L125.1,561.6 L125.6,561.6 L126.1,561.7 L126.6,561.8 L127.1,561.8 L127.6,561.9 L128.1,562.0 L128.6,562.1
 L129.1,562.1 L129.6,562.2 L130.1,562.3 L130.6,562.4 L131.1,562.4 L131.6,562.5 L132.2,562.6 L132.7,562.6
 L133.2,562.7 L133.7,562.8 L134.2,562.8 L134.7,562.9 L135.2,563.0 L135.7,563.0 L136.2,563.1 L136.7,563.1
 L137.2,563.2 L137.7,563.2 L138.2,563.3 L138.7,563.3 L139.2,563.4 L139.7,563.4 L140.2,563.5 L140.7,563.5
 L141.2,563.5 L141.7,563.6 L142.2,563.6 L142.7,563.7 L143.2,563.7 L143.7,563.7 L144.2,563.7 L144.7,563.8
 L145.2,563.8 L145.7,563.8 L146.2,563.8 L146.7,563.8 L147.2,563.9 L147.8,563.9 L148.3,563.9 L148.8,563.9
 L149.3,563.9 L149.8,563.9 L150.3,563.9 L150.8,563.9 L151.3,563.9 L151.8,563.9 L152.3,563.9 L152.8,563.9
 L153.3,563.9 L153.8,563.9 L154.3,563.9 L154.8,563.8 L155.3,563.8 L155.8,563.8 L156.3,563.8 L156.8,563.8
 L157.3,563.7 L157.8,563.7 L158.3,563.7 L158.8,563.7 L159.3,563.7 L159.8,563.6 L160.3,563.6 L160.8,563.6
 L161.3,563.6 L161.8,563.5 L162.3,563.5 L162.8,563.5 L163.4,563.5 L163.9,563.4 L164.4,563.4 L164.9,563.4
 L165.4,563.4 L165.9,563.3 L166.4,563.3 L166.9,563.3 L167.4,563.3 L167.9,563.3 L168.4,563.2 L168.9,563.2
 L169.4,

In [ ]:
var gp = PlotTableEnrgy[3,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 
 totalEnergyJ686 
 
 
 totalEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M410.9,81.1 L464.3,81.1 M45.6,508.9 L46.1,508.9 L46.6,508.9 L47.1,508.9 L47.6,508.9 L48.1,508.9
 L48.6,508.9 L49.1,508.9 L49.6,508.9 L50.1,508.9 L50.6,509.0 L51.1,509.0 L51.6,509.0 L52.1,509.1
 L52.6,509.1 L53.1,509.2 L53.7,509.2 L54.2,509.3 L54.7,509.3 L55.2,509.4 L55.7,509.4 L56.2,509.5
 L56.7,509.5 L57.2,509.6 L57.7,509.7 L58.2,509.8 L58.7,509.8 L59.2,509.9 L59.7,510.0 L60.2,510.1
 L60.7,510.2 L61.2,510.3 L61.7,510.4 L62.2,510.4 L62.7,510.5 L63.2,510.6 L63.7,510.7 L64.2,510.9
 L64.7,511.0 L65.2,511.1 L65.7,511.2 L66.2,511.3 L66.7,511.4 L67.2,511.5 L67.7,511.6 L68.2,511.7
 L68.7,511.9 L69.3,512.0 L69.8,512.1 L70.3,512.2 L70.8,512.3 L71.3,512.4 L71.8,512.6 L72.3,512.7
 L72.8,512.8 L73.3,512.9 L73.8,513.0 L74.3,513.2 L74.8,513.3 L75.3,513.4 L75.8,513.5 L76.3,513.6
 L76.8,513.8 L77.3,513.9 L77.8,514.0 L78.3,514.1 L78.8,514.2 L79.3,514.3 L79.8,514.4 L80.3,514.6
 L80.8,514.7 L81.3,514.8 L81.8,514.9 L82.3,515.0 L82.8,515.1 L83.3,515.2 L83.8,515.3 L84.3,515.4
 L84.9,515.5 L85.4,515.6 L85.9,515.7 L86.4,515.8 L86.9,515.9 L87.4,516.0 L87.9,516.1 L88.4,516.2
 L88.9,516.2 L89.4,516.3 L89.9,516.4 L90.4,516.5 L90.9,516.5 L91.4,516.6 L91.9,516.7 L92.4,516.7
 L92.9,516.8 L93.4,516.9 L93.9,516.9 L94.4,517.0 L94.9,517.1 L95.4,517.1 L95.9,517.2 L96.4,517.2
 L96.9,517.2 L97.4,517.3 L97.9,517.3 L98.4,517.4 L98.9,517.4 L99.4,517.4 L99.9,517.5 L100.5,517.5
 L101.0,517.5 L101.5,517.5 L102.0,517.5 L102.5,517.6 L103.0,517.6 L103.5,517.6 L104.0,517.6 L104.5,517.6
 L105.0,517.6 L105.5,517.6 L106.0,517.7 L106.5,517.7 L107.0,517.7 L107.5,517.7 L108.0,517.7 L108.5,517.7
 L109.0,517.7 L109.5,517.7 L110.0,517.7 L110.5,517.7 L111.0,517.7 L111.5,517.7 L112.0,517.7 L112.5,517.7
 L113.0,517.8 L113.5,517.8 L114.0,517.7 L114.5,517.7 L115.0,517.8 L115.5,517.8 L116.1,517.8 L116.6,517.8
 L117.1,517.8 L117.6,517.8 L118.1,517.8 L118.6,517.8 L119.1,517.8 L119.6,517.8 L120.1,517.9 L120.6,517.9
 L121.1,517.9 L121.6,517.9 L122.1,517.9 L122.6,517.9 L123.1,518.0 L123.6,518.0 L124.1,518.0 L124.6,518.0
 L125.1,518.0 L125.6,518.1 L126.1,518.1 L126.6,518.1 L127.1,518.1 L127.6,518.2 L128.1,518.2 L128.6,518.2
 L129.1,518.2 L129.6,518.3 L130.1,518.3 L130.6,518.3 L131.1,518.4 L131.6,518.4 L132.2,518.4 L132.7,518.4
 L133.2,518.5 L133.7,518.5 L134.2,518.5 L134.7,518.6 L135.2,518.6 L135.7,518.6 L136.2,518.7 L136.7,518.7
 L137.2,518.7 L137.7,518.8 L138.2,518.8 L138.7,518.8 L139.2,518.8 L139.7,518.9 L140.2,518.9 L140.7,518.9
 L141.2,519.0 L141.7,519.0 L142.2,519.0 L142.7,519.1 L143.2,519.1 L143.7,519.1 L144.2,519.2 L144.7,519.2
 L145.2,519.2 L145.7,519.3 L146.2,519.3 L146.7,519.3 L147.2,519.4 L147.8,519.4 L148.3,519.4 L148.8,519.5
 L149.3,519.5 L149.8,519.5 L150.3,519.5 L150.8,519.6 L151.3,519.6 L151.8,519.6 L152.3,519.6 L152.8,519.7
 L153.3,519.7 L153.8,519.7 L154.3,519.7 L154.8,519.7 L155.3,519.8 L155.8,519.8 L156.3,519.8 L156.8,519.8
 L157.3,519.8 L157.8,519.9 L158.3,519.9 L158.8,519.9 L159.3,519.9 L159.8,519.9 L160.3,519.9 L160.8,519.9
 L161.3,519.9 L161.8,520.0 L162.3,520.0 L162.8,520.0 L163.4,520.0 L163.9,520.0 L164.4,520.0 L164.9,520.0
 L165.4,520.0 L165.9,520.0 L166.4,520.0 L166.9,520.0 L167.4,520.0 L167.9,520.0 L168.4,520.0 L168.9,520.0
 L169.4,520.0 L169.9,520.0 L170.4,520.0 L170.9,520.0 L171.4,520.0 

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[3,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m3/" + caseS[3] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 5 (m3_Oh01_eta03)

In [ ]:
var gp = PlotTableEnrgy[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.05 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0.5 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy J686-3OrdInit 
 
 
 
 
 surfaceEnergy J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M377.6,57.1 L431.0,57.1 M62.2,334.8 L62.7,329.5 L63.2,321.1 L63.7,317.1 L64.2,311.4 L64.7,306.0
 L65.1,299.4 L65.6,294.8 L66.1,292.8 L66.6,296.0 L67.1,290.8 L67.6,290.2 L68.1,292.9 L68.6,292.8
 L69.1,290.4 L69.6,290.8 L70.1,292.5 L70.6,291.6 L71.0,294.6 L71.5,294.6 L72.0,296.2 L72.5,295.2
 L73.0,298.2 L73.5,299.2 L74.0,301.3 L74.5,307.8 L75.0,308.7 L75.5,308.6 L76.0,306.9 L76.4,311.8
 L76.9,313.4 L77.4,315.0 L77.9,318.8 L78.4,320.9 L78.9,323.8 L79.4,329.9 L79.9,331.6 L80.4,334.7
 L80.9,334.6 L81.4,339.1 L81.9,343.7 L82.3,344.7 L82.8,347.4 L83.3,349.8 L83.8,352.4 L84.3,354.8
 L84.8,356.9 L85.3,359.1 L85.8,363.9 L86.3,365.5 L86.8,369.1 L87.3,370.8 L87.8,375.2 L88.2,377.5
 L88.7,380.3 L89.2,385.0 L89.7,389.0 L90.2,387.1 L90.7,390.2 L91.2,393.3 L91.7,396.4 L92.2,399.3
 L92.7,402.1 L93.2,404.9 L93.6,407.5 L94.1,410.3 L94.6,413.2 L95.1,416.3 L95.6,419.1 L96.1,422.0
 L96.6,424.8 L97.1,427.6 L97.6,430.5 L98.1,433.3 L98.6,436.1 L99.1,438.8 L99.5,441.6 L100.0,444.3
 L100.5,447.1 L101.0,449.8 L101.5,452.5 L102.0,455.4 L102.5,458.2 L103.0,461.0 L103.5,463.6 L104.0,466.2
 L104.5,468.7 L104.9,471.1 L105.4,473.5 L105.9,475.8 L106.4,478.1 L106.9,480.3 L107.4,482.3 L107.9,484.2
 L108.4,485.9 L108.9,487.5 L109.4,489.0 L109.9,490.3 L110.4,491.6 L110.8,492.6 L111.3,493.5 L111.8,494.3
 L112.3,494.9 L112.8,495.6 L113.3,495.9 L113.8,496.1 L114.3,496.2 L114.8,496.1 L115.3,496.0 L115.8,495.7
 L116.2,495.4 L116.7,494.9 L117.2,494.3 L117.7,493.7 L118.2,493.1 L118.7,492.3 L119.2,491.5 L119.7,490.6
 L120.2,489.8 L120.7,488.8 L121.2,487.9 L121.7,487.0 L122.1,486.0 L122.6,485.0 L123.1,484.1 L123.6,483.1
 L124.1,482.2 L124.6,481.3 L125.1,480.4 L125.6,479.5 L126.1,478.6 L126.6,477.9 L127.1,477.1 L127.6,476.4
 L128.0,475.7 L128.5,475.0 L129.0,474.5 L129.5,473.9 L130.0,473.4 L130.5,473.0 L131.0,472.6 L131.5,472.3
 L132.0,472.0 L132.5,471.8 L133.0,471.6 L133.4,471.5 L133.9,471.5 L134.4,471.4 L134.9,471.5 L135.4,471.6
 L135.9,471.7 L136.4,471.9 L136.9,472.1 L137.4,472.5 L137.9,472.9 L138.4,473.3 L138.9,473.8 L139.3,474.3
 L139.8,474.5 L140.3,475.0 L140.8,475.5 L141.3,476.1 L141.8,476.7 L142.3,477.3 L142.8,478.0 L143.3,478.7
 L143.8,479.4 L144.3,480.1 L144.7,480.8 L145.2,481.6 L145.7,482.4 L146.2,483.2 L146.7,484.0 L147.2,484.8
 L147.7,485.6 L148.2,486.6 L148.7,487.4 L149.2,488.2 L149.7,489.0 L150.2,489.9 L150.6,490.8 L151.1,491.7
 L151.6,492.5 L152.1,493.3 L152.6,494.1 L153.1,495.0 L153.6,495.8 L154.1,496.6 L154.6,497.5 L155.1,498.2
 L155.6,499.0 L156.0,499.8 L156.5,500.5 L157.0,501.3 L157.5,502.1 L158.0,502.8 L158.5,503.5 L159.0,504.1
 L159.5,504.8 L160.0,505.3 L160.5,505.9 L161.0,506.5 L161.5,507.0 L161.9,507.5 L162.4,508.0 L162.9,508.5
 L163.4,509.1 L163.9,509.5 L164.4,509.9 L164.9,510.2 L165.4,510.6 L165.9,510.9 L166.4,511.2 L166.9,511.5
 L167.4,511.7 L167.8,511.9 L168.3,512.1 L168.8,512.3 L169.3,512.5 L169.8,512.6 L170.3,512.7 L170.8,512.8
 L171.3,512.9 L171.8,512.9 L172.3,512.9 L172.8,512.9 L173.2,512.9 L173.7,512.9 L174.2,512.9 L174.7,512.9
 L175.2,512.8 L175.7,512.7 L176.2,512.6 L176.7,512.5 L177.2,512.4 L177.7,512.2 L178.2,512.1 L178.7,511.9
 L179.1,511.8 L179.6,511.6 L180.1,511.5 L1

In [ ]:
var gp = PlotTableEnrgy[4,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.1 
 
 
 
 
 3.2 
 
 
 
 
 3.3 
 
 
 
 
 3.4 
 
 
 
 
 3.5 
 
 
 
 
 3.6 
 
 
 
 
 3.7 
 
 
 
 
 3.8 
 
 
 
 
 3.9 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy J686-3OrdInit 
 
 
 
 
 totalEnergy J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M399.8,57.1 L453.2,57.1 M53.9,85.0 L54.4,178.2 L54.9,234.9 L55.4,266.4 L55.9,284.2 L56.4,301.1
 L56.9,310.3 L57.4,316.8 L57.9,323.0 L58.4,331.1 L58.9,332.4 L59.4,336.1 L59.9,341.1 L60.4,343.5
 L60.9,343.9 L61.4,345.9 L61.9,348.4 L62.4,348.9 L62.9,351.7 L63.3,352.4 L63.8,353.9 L64.3,353.4
 L64.8,355.6 L65.3,356.3 L65.8,357.6 L66.3,361.7 L66.8,362.1 L67.3,361.6 L67.8,360.0 L68.3,362.9
 L68.8,363.4 L69.3,363.9 L69.8,365.8 L70.3,366.6 L70.8,367.8 L71.3,371.2 L71.8,371.5 L72.3,372.9
 L72.8,372.2 L73.3,374.4 L73.8,376.7 L74.3,376.6 L74.8,377.6 L75.3,378.5 L75.8,379.5 L76.3,380.3
 L76.8,380.9 L77.3,381.6 L77.8,384.1 L78.3,384.4 L78.8,386.1 L79.3,386.6 L79.8,388.9 L80.3,389.8
 L80.8,391.0 L81.3,393.6 L81.7,395.7 L82.2,393.9 L82.7,395.5 L83.2,397.2 L83.7,398.8 L84.2,400.4
 L84.7,402.0 L85.2,403.6 L85.7,405.1 L86.2,406.7 L86.7,408.4 L87.2,410.3 L87.7,411.9 L88.2,413.7
 L88.7,415.4 L89.2,417.1 L89.7,418.9 L90.2,420.7 L90.7,422.4 L91.2,424.1 L91.7,425.9 L92.2,427.6
 L92.7,429.4 L93.2,431.2 L93.7,433.0 L94.2,434.9 L94.7,436.8 L95.2,438.7 L95.7,440.6 L96.2,442.5
 L96.7,444.4 L97.2,446.3 L97.7,448.3 L98.2,450.2 L98.7,452.2 L99.2,454.2 L99.7,456.1 L100.1,458.1
 L100.6,459.9 L101.1,461.8 L101.6,463.6 L102.1,465.4 L102.6,467.2 L103.1,468.9 L103.6,470.6 L104.1,472.2
 L104.6,473.8 L105.1,475.5 L105.6,476.9 L106.1,478.3 L106.6,479.6 L107.1,480.9 L107.6,482.2 L108.1,483.4
 L108.6,484.6 L109.1,485.7 L109.6,486.7 L110.1,487.8 L110.6,488.7 L111.1,489.6 L111.6,490.4 L112.1,491.2
 L112.6,491.9 L113.1,492.6 L113.6,493.2 L114.1,493.8 L114.6,494.4 L115.1,494.9 L115.6,495.4 L116.1,495.9
 L116.6,496.3 L117.1,496.7 L117.6,497.1 L118.0,497.4 L118.5,497.7 L119.0,498.0 L119.5,498.3 L120.0,498.6
 L120.5,498.8 L121.0,499.1 L121.5,499.3 L122.0,499.5 L122.5,499.7 L123.0,499.8 L123.5,500.0 L124.0,500.2
 L124.5,500.3 L125.0,500.5 L125.5,500.6 L126.0,500.8 L126.5,500.9 L127.0,501.1 L127.5,501.2 L128.0,501.3
 L128.5,501.5 L129.0,501.6 L129.5,501.7 L130.0,501.9 L130.5,502.1 L131.0,502.3 L131.5,502.5 L132.0,502.7
 L132.5,502.7 L133.0,502.8 L133.5,503.0 L134.0,503.1 L134.5,503.3 L135.0,503.5 L135.5,503.7 L136.0,503.9
 L136.4,504.1 L136.9,504.2 L137.4,504.4 L137.9,504.7 L138.4,504.9 L138.9,505.1 L139.4,505.3 L139.9,505.5
 L140.4,505.8 L140.9,506.1 L141.4,506.3 L141.9,506.5 L142.4,506.8 L142.9,507.0 L143.4,507.4 L143.9,507.6
 L144.4,507.9 L144.9,508.1 L145.4,508.4 L145.9,508.7 L146.4,509.0 L146.9,509.3 L147.4,509.6 L147.9,509.9
 L148.4,510.2 L148.9,510.5 L149.4,510.8 L149.9,511.1 L150.4,511.5 L150.9,511.8 L151.4,512.1 L151.9,512.3
 L152.4,512.7 L152.9,513.0 L153.4,513.2 L153.9,513.5 L154.4,513.8 L154.8,514.1 L155.3,514.4 L155.8,514.7
 L156.3,515.1 L156.8,515.4 L157.3,515.7 L157.8,516.0 L158.3,516.2 L158.8,516.5 L159.3,516.8 L159.8,517.1
 L160.3,517.3 L160.8,517.6 L161.3,517.9 L161.8,518.1 L162.3,518.4 L162.8,518.6 L163.3,518.9 L163.8,519.1
 L164.3,519.3 L164.8,519.5 L165.3,519.8 L165.8,520.0 L166.3,520.2 L166.8,520.4 L167.3,520.6 L167.8,520.8
 L168.3,521.0 L168.8,521.2 L169.3,521.3 L169.8,521.5 L170.3,521.7 L170.8,521.8 L171.3,522.0 L171.8,522.1
 L172.3,522.3 L172.8,522.4 L173.2,522.6 L173.7,522.7 L174.2,522.8 L174.7,522.9 L175.2,523.0 L175.7,523.1
 L176.2,523.2 L176.7,523.3 L177.2,5

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[4,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m3/" + caseS[4] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 6 (m3_Oh01_eta015)

In [ ]:
var gp = PlotTableEnrgy[5,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0.04 
 
 
 
 
 0.045 
 
 
 
 
 0.05 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M70.5,39.8 L71.0,39.4 L71.5,39.9 L72.0,41.0 L72.4,42.8 L72.9,45.2
 L73.4,48.3 L73.9,52.0 L74.4,56.2 L74.9,61.0 L75.4,66.3 L75.8,72.2 L76.3,78.5 L76.8,85.4
 L77.3,92.6 L77.8,100.3 L78.3,108.4 L78.8,116.8 L79.2,125.6 L79.7,134.6 L80.2,143.9 L80.7,153.6
 L81.2,163.4 L81.7,173.4 L82.2,183.6 L82.6,194.0 L83.1,204.5 L83.6,214.9 L84.1,225.7 L84.6,236.4
 L85.1,247.1 L85.5,257.9 L86.0,268.7 L86.5,279.4 L87.0,290.1 L87.5,300.6 L88.0,311.1 L88.5,321.6
 L88.9,331.7 L89.4,341.9 L89.9,351.9 L90.4,361.5 L90.9,371.2 L91.4,380.6 L91.9,389.7 L92.3,398.6
 L92.8,407.2 L93.3,415.5 L93.8,423.1 L94.3,430.9 L94.8,438.4 L95.3,445.3 L95.7,452.2 L96.2,458.6
 L96.7,464.5 L97.2,470.3 L97.7,475.8 L98.2,480.9 L98.7,485.4 L99.1,489.9 L99.6,493.6 L100.1,497.4
 L100.6,500.4 L101.1,503.5 L101.6,505.8 L102.1,507.9 L102.5,509.7 L103.0,511.3 L103.5,512.5 L104.0,513.3
 L104.5,513.8 L105.0,513.9 L105.5,513.7 L105.9,513.3 L106.4,512.5 L106.9,511.5 L107.4,510.0 L107.9,508.5
 L108.4,506.8 L108.8,504.6 L109.3,502.4 L109.8,500.1 L110.3,497.5 L110.8,494.8 L111.3,491.7 L111.8,488.7
 L112.2,485.5 L112.7,482.0 L113.2,478.7 L113.7,475.0 L114.2,471.5 L114.7,467.9 L115.2,464.2 L115.6,460.2
 L116.1,456.6 L116.6,452.8 L117.1,449.1 L117.6,445.4 L118.1,441.6 L118.6,438.0 L119.0,434.3 L119.5,430.8
 L120.0,427.3 L120.5,423.6 L121.0,420.3 L121.5,417.3 L122.0,414.3 L122.4,411.4 L122.9,408.4 L123.4,405.9
 L123.9,403.4 L124.4,401.1 L124.9,398.9 L125.4,396.8 L125.8,394.9 L126.3,393.2 L126.8,391.7 L127.3,390.3
 L127.8,389.1 L128.3,388.1 L128.8,387.3 L129.2,386.7 L129.7,386.2 L130.2,386.0 L130.7,385.9 L131.2,386.0
 L131.7,386.2 L132.1,386.7 L132.6,387.3 L133.1,388.1 L133.6,389.0 L134.1,390.1 L134.6,391.4 L135.1,392.8
 L135.5,394.4 L136.0,396.1 L136.5,397.9 L137.0,399.8 L137.5,401.9 L138.0,404.1 L138.5,406.4 L138.9,408.8
 L139.4,411.3 L139.9,413.9 L140.4,416.2 L140.9,419.1 L141.4,422.0 L141.9,424.8 L142.3,427.5 L142.8,430.4
 L143.3,433.5 L143.8,436.5 L144.3,439.5 L144.8,442.5 L145.3,445.6 L145.7,448.6 L146.2,451.6 L146.7,454.6
 L147.2,457.6 L147.7,460.4 L148.2,463.4 L148.7,466.3 L149.1,469.1 L149.6,471.9 L150.1,474.6 L150.6,477.3
 L151.1,479.9 L151.6,482.4 L152.1,484.9 L152.5,487.3 L153.0,489.6 L153.5,491.8 L154.0,493.9 L154.5,496.0
 L155.0,497.9 L155.5,499.8 L155.9,501.5 L156.4,503.2 L156.9,504.7 L157.4,506.2 L157.9,507.5 L158.4,508.8
 L158.8,509.9 L159.3,511.0 L159.8,511.9 L160.3,512.7 L160.8,513.5 L161.3,514.1 L161.8,514.7 L162.2,515.1
 L162.7,515.4 L163.2,515.7 L163.7,515.8 L164.2,515.9 L164.7,515.8 L165.2,515.7 L165.6,515.5 L166.1,515.2
 L166.6,514.9 L167.1,514.5 L167.6,514.0 L168.1,513.4 L168.6,512.8 L169.0,512.1 L169.5,511.5 L170.0,510.7
 L170.5,509.9 L171.0,509.0 L171.5,508.1 L172.0,507.2 L172.4,506.2 L172.9,505.2 L173.4,504.2 L173.9,503.1
 L174.4,502.1 L174.9,501.0 L175.4,499.9 L175.8,498.8 L176.3,497.8 L176.8,496.7 L177.3,495.7 L177.8,494.6
 L178.3,493.5 L178.8,492.5 L179.2,491.5 L179.7,490.5 L180.2,489.6 L180.7,488.7 L181.2,487.8 L181.7,486.9
 L182.1,486.1 L182.6,485.3 L183.1,484.6 L183.6,483.9 L184.1,483.3 L184.6,482.7 L185.1,482.2 L185.5,481.7
 L186.0,481.2 L186.5,480.8 L187.

In [ ]:
var gp = PlotTableEnrgy[5,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.15 
 
 
 
 
 3.16 
 
 
 
 
 3.17 
 
 
 
 
 3.18 
 
 
 
 
 3.19 
 
 
 
 
 3.2 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M62.2,95.0 L62.7,95.8 L63.2,96.8 L63.7,97.6 L64.2,98.5 L64.7,99.4
 L65.1,100.4 L65.6,101.4 L66.1,102.5 L66.6,103.7 L67.1,105.1 L67.6,106.5 L68.1,108.1 L68.6,109.8
 L69.1,111.6 L69.6,113.5 L70.1,115.6 L70.6,117.8 L71.0,120.1 L71.5,122.5 L72.0,125.0 L72.5,127.7
 L73.0,130.6 L73.5,133.5 L74.0,136.6 L74.5,139.7 L75.0,143.0 L75.5,146.3 L76.0,149.9 L76.4,153.6
 L76.9,157.2 L77.4,161.1 L77.9,165.1 L78.4,169.2 L78.9,173.4 L79.4,177.5 L79.9,181.9 L80.4,186.3
 L80.9,190.7 L81.4,195.4 L81.9,200.1 L82.3,204.7 L82.8,209.6 L83.3,214.5 L83.8,219.4 L84.3,224.3
 L84.8,229.3 L85.3,234.3 L85.8,239.0 L86.3,244.1 L86.8,249.3 L87.3,254.1 L87.8,259.3 L88.2,264.4
 L88.7,269.3 L89.2,274.3 L89.7,279.5 L90.2,284.6 L90.7,289.3 L91.2,294.4 L91.7,299.1 L92.2,304.1
 L92.7,308.7 L93.2,313.5 L93.6,318.0 L94.1,322.5 L94.6,326.9 L95.1,331.5 L95.6,335.8 L96.1,340.0
 L96.6,344.1 L97.1,348.1 L97.6,352.1 L98.1,355.9 L98.6,359.6 L99.1,363.2 L99.5,366.4 L100.0,369.9
 L100.5,373.2 L101.0,376.3 L101.5,379.3 L102.0,382.3 L102.5,385.2 L103.0,387.9 L103.5,390.3 L104.0,392.9
 L104.5,395.3 L104.9,397.4 L105.4,399.7 L105.9,401.6 L106.4,403.6 L106.9,405.5 L107.4,407.2 L107.9,408.6
 L108.4,410.3 L108.9,411.7 L109.4,413.1 L109.9,414.3 L110.4,415.4 L110.8,416.5 L111.3,417.5 L111.8,418.4
 L112.3,419.2 L112.8,419.7 L113.3,420.4 L113.8,421.2 L114.3,421.8 L114.8,422.3 L115.3,422.6 L115.8,423.2
 L116.2,423.6 L116.7,423.9 L117.2,424.2 L117.7,424.5 L118.2,424.7 L118.7,424.9 L119.2,425.1 L119.7,425.2
 L120.2,425.4 L120.7,425.5 L121.2,425.7 L121.7,425.8 L122.1,425.9 L122.6,426.1 L123.1,426.2 L123.6,426.4
 L124.1,426.6 L124.6,426.7 L125.1,426.9 L125.6,427.2 L126.1,427.4 L126.6,427.7 L127.1,427.9 L127.6,428.3
 L128.0,428.6 L128.5,429.0 L129.0,429.4 L129.5,429.9 L130.0,430.3 L130.5,430.9 L131.0,431.4 L131.5,432.0
 L132.0,432.6 L132.5,433.3 L133.0,433.7 L133.4,434.6 L133.9,435.4 L134.4,436.3 L134.9,436.9 L135.4,437.8
 L135.9,438.8 L136.4,439.8 L136.9,440.8 L137.4,441.8 L137.9,442.9 L138.4,443.9 L138.9,445.1 L139.3,446.2
 L139.8,447.4 L140.3,448.5 L140.8,449.7 L141.3,451.0 L141.8,452.3 L142.3,453.6 L142.8,454.9 L143.3,456.2
 L143.8,457.6 L144.3,458.9 L144.7,460.3 L145.2,461.7 L145.7,463.0 L146.2,464.5 L146.7,465.9 L147.2,467.3
 L147.7,468.7 L148.2,470.1 L148.7,471.5 L149.2,472.9 L149.7,474.2 L150.2,475.6 L150.6,477.0 L151.1,478.3
 L151.6,479.7 L152.1,481.1 L152.6,482.4 L153.1,483.6 L153.6,484.9 L154.1,486.2 L154.6,487.4 L155.1,488.6
 L155.6,489.8 L156.0,491.0 L156.5,492.1 L157.0,493.2 L157.5,494.3 L158.0,495.4 L158.5,496.4 L159.0,497.4
 L159.5,498.3 L160.0,499.3 L160.5,500.2 L161.0,501.1 L161.5,501.9 L161.9,502.7 L162.4,503.5 L162.9,504.3
 L163.4,505.0 L163.9,505.7 L164.4,506.4 L164.9,507.0 L165.4,507.6 L165.9,508.1 L166.4,508.7 L166.9,509.2
 L167.4,509.6 L167.8,510.1 L168.3,510.5 L168.8,510.9 L169.3,511.2 L169.8,511.6 L170.3,511.9 L170.8,512.2
 L171.3,512.5 L171.8,512.7 L172.3,512.9 L172.8,513.1 L173.2,513.3 L173.7,513.5 L174.2,513.6 L174.7,513.8
 L175.2,513.9 L175.7,514.0 L176.2,514.1 L176.7,514.2 L177.2,514.3 L177.7,514.4 L178.2,514.5 L178.7,514.5
 L179.1,514.6 L179.6,514.6 L180.1,514.6 L180.6,514.7 L181.1,514.7 L181.6,514.7 L182.1,514.8 L182.6,514.8
 L183.1,514.8 L183.6,514.9 L184.1,515.0 L184.5,515.2 L185.0,515.2 L1

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[5,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m3/" + caseS[5] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 7 (m4_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[6,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.05 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686 
 
 
 
 
 surfaceEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M62.2,47.9 L62.7,47.9 L63.2,49.1 L63.7,51.4 L64.2,54.7 L64.7,58.9
 L65.1,63.9 L65.6,69.7 L66.1,76.2 L66.6,83.4 L67.1,91.2 L67.6,99.6 L68.1,108.4 L68.6,117.8
 L69.1,127.6 L69.6,137.8 L70.1,148.3 L70.6,159.2 L71.0,170.4 L71.5,181.8 L72.0,193.4 L72.5,205.2
 L73.0,217.1 L73.5,229.0 L74.0,241.0 L74.5,253.1 L75.0,265.0 L75.5,277.0 L76.0,288.9 L76.4,300.6
 L76.9,312.2 L77.4,323.6 L77.9,334.8 L78.4,345.8 L78.9,356.5 L79.4,366.9 L79.9,377.0 L80.4,386.8
 L80.9,396.2 L81.4,405.3 L81.9,414.0 L82.3,422.2 L82.8,430.0 L83.3,437.3 L83.8,444.2 L84.3,450.6
 L84.8,456.6 L85.3,462.1 L85.8,467.1 L86.3,472.1 L86.8,476.2 L87.3,480.0 L87.8,483.3 L88.2,486.0
 L88.7,488.3 L89.2,490.1 L89.7,491.8 L90.2,493.3 L90.7,494.1 L91.2,494.3 L91.7,494.1 L92.2,493.5
 L92.7,492.5 L93.2,491.1 L93.6,489.3 L94.1,487.2 L94.6,485.0 L95.1,482.3 L95.6,479.4 L96.1,476.3
 L96.6,472.9 L97.1,469.3 L97.6,465.6 L98.1,461.8 L98.6,457.9 L99.1,454.0 L99.5,450.1 L100.0,446.3
 L100.5,442.7 L101.0,439.1 L101.5,435.6 L102.0,432.2 L102.5,429.1 L103.0,426.2 L103.5,423.6 L104.0,421.2
 L104.5,419.0 L104.9,417.2 L105.4,415.6 L105.9,414.4 L106.4,413.5 L106.9,412.9 L107.4,412.7 L107.9,412.7
 L108.4,413.0 L108.9,413.6 L109.4,414.5 L109.9,415.7 L110.4,417.1 L110.8,418.7 L111.3,420.5 L111.8,422.5
 L112.3,424.7 L112.8,427.1 L113.3,429.8 L113.8,432.5 L114.3,435.1 L114.8,438.1 L115.3,441.0 L115.8,443.9
 L116.2,446.8 L116.7,449.7 L117.2,452.7 L117.7,455.7 L118.2,458.7 L118.7,461.7 L119.2,464.5 L119.7,467.3
 L120.2,470.0 L120.7,472.7 L121.2,475.3 L121.7,477.7 L122.1,480.1 L122.6,482.4 L123.1,484.6 L123.6,486.8
 L124.1,488.8 L124.6,490.6 L125.1,492.3 L125.6,493.8 L126.1,495.3 L126.6,496.5 L127.1,497.7 L127.6,498.7
 L128.0,499.6 L128.5,500.6 L129.0,501.2 L129.5,501.7 L130.0,502.1 L130.5,502.5 L131.0,502.6 L131.5,502.7
 L132.0,502.9 L132.5,502.8 L133.0,502.6 L133.4,502.4 L133.9,502.1 L134.4,501.7 L134.9,501.2 L135.4,500.7
 L135.9,500.1 L136.4,499.5 L136.9,498.9 L137.4,498.3 L137.9,497.6 L138.4,496.9 L138.9,496.2 L139.3,495.8
 L139.8,495.1 L140.3,494.5 L140.8,493.8 L141.3,493.1 L141.8,492.4 L142.3,491.8 L142.8,491.3 L143.3,490.7
 L143.8,490.3 L144.3,489.8 L144.7,489.4 L145.2,489.1 L145.7,488.8 L146.2,488.5 L146.7,488.3 L147.2,488.2
 L147.7,488.1 L148.2,488.0 L148.7,488.1 L149.2,488.1 L149.7,488.2 L150.2,488.4 L150.6,488.6 L151.1,488.9
 L151.6,489.2 L152.1,489.6 L152.6,490.0 L153.1,490.4 L153.6,490.9 L154.1,491.4 L154.6,491.9 L155.1,492.5
 L155.6,493.1 L156.0,493.7 L156.5,494.4 L157.0,495.0 L157.5,495.7 L158.0,496.8 L158.5,497.5 L159.0,498.1
 L159.5,498.8 L160.0,499.5 L160.5,500.2 L161.0,501.0 L161.5,501.6 L161.9,502.3 L162.4,502.9 L162.9,503.5
 L163.4,504.1 L163.9,504.7 L164.4,505.3 L164.9,505.9 L165.4,506.4 L165.9,507.1 L166.4,507.5 L166.9,508.0
 L167.4,508.4 L167.8,508.7 L168.3,509.1 L168.8,509.4 L169.3,509.6 L169.8,509.9 L170.3,510.1 L170.8,510.3
 L171.3,510.4 L171.8,510.5 L172.3,510.6 L172.8,510.7 L173.2,510.7 L173.7,510.7 L174.2,510.7 L174.7,510.7
 L175.2,510.6 L175.7,510.5 L176.2,510.7 L176.7,510.6 L177.2,510.5 L177.7,510.5 L178.2,510.4 L178.7,510.2
 L179.1,510.1 L179.6,509.9 L180.1,509.7 L180.6,509.5 L181.1,509.3 L181.6,509.1 L182.1,508.9 L182.6,508.7
 L183.1,508.6 L183.6,

In [ ]:
var gp = PlotTableEnrgy[6,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.1 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 3.55 
 
 
 
 
 3.6 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686 
 
 
 
 
 totalEnergyJ686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M510.8,57.1 L564.2,57.1 M62.2,56.8 L62.7,56.3 L63.2,55.9 L63.7,55.7 L64.2,55.8 L64.7,56.1
 L65.1,56.7 L65.6,57.5 L66.1,58.6 L66.6,59.9 L67.1,61.5 L67.6,63.3 L68.1,65.3 L68.6,67.6
 L69.1,70.1 L69.6,72.8 L70.1,75.8 L70.6,79.0 L71.0,82.5 L71.5,86.2 L72.0,90.1 L72.5,94.2
 L73.0,98.6 L73.5,103.1 L74.0,107.9 L74.5,112.8 L75.0,117.9 L75.5,123.3 L76.0,128.7 L76.4,134.4
 L76.9,140.2 L77.4,146.2 L77.9,152.3 L78.4,158.6 L78.9,164.9 L79.4,171.4 L79.9,178.0 L80.4,184.7
 L80.9,191.5 L81.4,198.3 L81.9,205.2 L82.3,212.1 L82.8,219.0 L83.3,225.9 L83.8,232.9 L84.3,239.8
 L84.8,246.7 L85.3,253.6 L85.8,260.4 L86.3,267.7 L86.8,274.5 L87.3,281.3 L87.8,288.0 L88.2,294.6
 L88.7,301.0 L89.2,307.4 L89.7,313.9 L90.2,320.6 L90.7,326.8 L91.2,332.7 L91.7,338.6 L92.2,344.3
 L92.7,349.8 L93.2,355.0 L93.6,360.1 L94.1,364.9 L94.6,369.8 L95.1,374.2 L95.6,378.5 L96.1,382.4
 L96.6,386.0 L97.1,389.4 L97.6,392.4 L98.1,395.2 L98.6,397.8 L99.1,400.1 L99.5,402.1 L100.0,404.0
 L100.5,405.7 L101.0,407.2 L101.5,408.4 L102.0,409.4 L102.5,410.3 L103.0,411.0 L103.5,411.8 L104.0,412.3
 L104.5,412.7 L104.9,413.1 L105.4,413.4 L105.9,413.8 L106.4,414.1 L106.9,414.4 L107.4,414.8 L107.9,415.1
 L108.4,415.5 L108.9,416.0 L109.4,416.5 L109.9,417.0 L110.4,417.6 L110.8,418.3 L111.3,419.0 L111.8,419.8
 L112.3,420.7 L112.8,421.7 L113.3,422.9 L113.8,424.1 L114.3,425.1 L114.8,426.5 L115.3,427.8 L115.8,429.1
 L116.2,430.5 L116.7,431.9 L117.2,433.3 L117.7,434.9 L118.2,436.5 L118.7,438.1 L119.2,439.7 L119.7,441.4
 L120.2,443.0 L120.7,444.7 L121.2,446.3 L121.7,448.0 L122.1,449.7 L122.6,451.4 L123.1,453.1 L123.6,455.0
 L124.1,456.6 L124.6,458.3 L125.1,459.9 L125.6,461.5 L126.1,463.0 L126.6,464.6 L127.1,466.0 L127.6,467.5
 L128.0,468.9 L128.5,470.5 L129.0,471.8 L129.5,473.1 L130.0,474.3 L130.5,475.5 L131.0,476.7 L131.5,477.8
 L132.0,479.0 L132.5,480.0 L133.0,481.0 L133.4,481.9 L133.9,482.8 L134.4,483.5 L134.9,484.3 L135.4,485.0
 L135.9,485.6 L136.4,486.2 L136.9,486.8 L137.4,487.3 L137.9,487.8 L138.4,488.3 L138.9,488.7 L139.3,489.3
 L139.8,489.6 L140.3,490.0 L140.8,490.2 L141.3,490.4 L141.8,490.6 L142.3,490.8 L142.8,491.0 L143.3,491.1
 L143.8,491.3 L144.3,491.4 L144.7,491.5 L145.2,491.6 L145.7,491.6 L146.2,491.7 L146.7,491.8 L147.2,491.8
 L147.7,491.9 L148.2,492.0 L148.7,492.0 L149.2,492.1 L149.7,492.2 L150.2,492.2 L150.6,492.3 L151.1,492.4
 L151.6,492.5 L152.1,492.6 L152.6,492.7 L153.1,492.9 L153.6,493.0 L154.1,493.2 L154.6,493.3 L155.1,493.5
 L155.6,493.7 L156.0,493.9 L156.5,494.2 L157.0,494.4 L157.5,494.7 L158.0,495.4 L158.5,495.6 L159.0,495.9
 L159.5,496.2 L160.0,496.5 L160.5,496.8 L161.0,497.3 L161.5,497.6 L161.9,498.0 L162.4,498.3 L162.9,498.7
 L163.4,499.1 L163.9,499.4 L164.4,499.8 L164.9,500.3 L165.4,500.7 L165.9,501.3 L166.4,501.7 L166.9,502.0
 L167.4,502.4 L167.8,502.8 L168.3,503.2 L168.8,503.6 L169.3,504.0 L169.8,504.4 L170.3,504.7 L170.8,505.1
 L171.3,505.5 L171.8,505.8 L172.3,506.1 L172.8,506.5 L173.2,506.8 L173.7,507.1 L174.2,507.4 L174.7,507.6
 L175.2,507.9 L175.7,508.1 L176.2,508.7 L176.7,508.9 L177.2,509.1 L177.7,509.5 L178.2,509.7 L178.7,509.9
 L179.1,510.0 L179.6,510.2 L180.1,510.3 L180.6,510.5 L181.1,510.6 L181.6,510.7 L182.1,510.8 L182.6,510.9
 L183.1,511.0 L183.6,511.0 L184.1,51

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[6,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m4/" + caseS[6] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 8 (m4_Oh01_eta01)

In [ ]:
var gp = PlotTableEnrgy[7,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M70.5,88.1 L71.0,87.8 L71.5,88.8 L72.0,91.1 L72.4,94.5 L72.9,99.0
 L73.4,104.4 L73.9,111.0 L74.4,118.5 L74.9,126.9 L75.4,136.1 L75.8,146.0 L76.3,156.7 L76.8,168.0
 L77.3,179.7 L77.8,192.1 L78.3,204.8 L78.8,218.0 L79.2,231.4 L79.7,245.0 L80.2,258.7 L80.7,272.5
 L81.2,286.3 L81.7,300.1 L82.2,313.8 L82.6,327.2 L83.1,340.5 L83.6,353.2 L84.1,365.9 L84.6,378.2
 L85.1,390.0 L85.5,401.3 L86.0,412.1 L86.5,422.3 L87.0,431.9 L87.5,440.9 L88.0,449.3 L88.5,457.0
 L88.9,464.0 L89.4,470.4 L89.9,476.0 L90.4,481.0 L90.9,485.3 L91.4,488.9 L91.9,491.8 L92.3,494.1
 L92.8,495.8 L93.3,496.8 L93.8,497.2 L94.3,497.1 L94.8,496.4 L95.3,495.3 L95.7,493.4 L96.2,491.5
 L96.7,489.1 L97.2,486.1 L97.7,483.1 L98.2,479.7 L98.7,476.1 L99.1,472.2 L99.6,468.2 L100.1,464.1
 L100.6,459.9 L101.1,455.7 L101.6,451.4 L102.1,447.2 L102.5,442.7 L103.0,438.9 L103.5,435.0 L104.0,431.3
 L104.5,427.7 L105.0,424.4 L105.5,421.3 L105.9,418.4 L106.4,415.8 L106.9,413.5 L107.4,411.5 L107.9,409.7
 L108.4,408.3 L108.8,407.2 L109.3,406.4 L109.8,405.9 L110.3,405.8 L110.8,405.9 L111.3,406.3 L111.8,407.1
 L112.2,408.1 L112.7,409.4 L113.2,410.9 L113.7,412.7 L114.2,414.7 L114.7,416.9 L115.2,419.3 L115.6,421.9
 L116.1,424.7 L116.6,427.5 L117.1,430.5 L117.6,433.6 L118.1,436.8 L118.6,440.0 L119.0,443.3 L119.5,446.5
 L120.0,449.6 L120.5,453.0 L121.0,456.3 L121.5,459.5 L122.0,462.6 L122.4,465.6 L122.9,468.6 L123.4,471.4
 L123.9,474.2 L124.4,476.8 L124.9,479.3 L125.4,481.6 L125.8,483.8 L126.3,485.8 L126.8,487.7 L127.3,489.4
 L127.8,491.0 L128.3,492.3 L128.8,493.5 L129.2,494.6 L129.7,495.5 L130.2,496.2 L130.7,496.8 L131.2,497.3
 L131.7,497.6 L132.1,497.8 L132.6,497.8 L133.1,497.7 L133.6,497.5 L134.1,497.2 L134.6,496.8 L135.1,496.3
 L135.5,495.7 L136.0,495.1 L136.5,494.4 L137.0,493.6 L137.5,492.8 L138.0,491.9 L138.5,491.0 L138.9,490.1
 L139.4,489.2 L139.9,488.3 L140.4,487.3 L140.9,486.4 L141.4,485.5 L141.9,484.7 L142.3,483.8 L142.8,483.0
 L143.3,482.2 L143.8,481.5 L144.3,480.8 L144.8,480.2 L145.3,479.7 L145.7,479.2 L146.2,478.7 L146.7,478.3
 L147.2,478.0 L147.7,477.8 L148.2,477.6 L148.7,477.5 L149.1,477.5 L149.6,477.5 L150.1,477.6 L150.6,477.7
 L151.1,477.9 L151.6,478.2 L152.1,478.5 L152.5,478.9 L153.0,479.3 L153.5,479.8 L154.0,480.3 L154.5,480.9
 L155.0,481.5 L155.5,482.1 L155.9,482.8 L156.4,483.5 L156.9,484.1 L157.4,484.9 L157.9,485.6 L158.4,486.3
 L158.8,487.1 L159.3,487.8 L159.8,488.5 L160.3,489.2 L160.8,489.9 L161.3,490.6 L161.8,491.3 L162.2,492.0
 L162.7,492.6 L163.2,493.2 L163.7,493.8 L164.2,494.4 L164.7,494.9 L165.2,495.4 L165.6,495.8 L166.1,496.3
 L166.6,496.6 L167.1,497.0 L167.6,497.3 L168.1,497.6 L168.6,497.8 L169.0,498.0 L169.5,498.2 L170.0,498.3
 L170.5,498.4 L171.0,498.5 L171.5,498.5 L172.0,498.5 L172.4,498.5 L172.9,498.5 L173.4,498.4 L173.9,498.3
 L174.4,498.2 L174.9,498.0 L175.4,497.9 L175.8,497.7 L176.3,497.5 L176.8,497.4 L177.3,497.2 L177.8,496.9
 L178.3,496.7 L178.8,496.5 L179.2,496.3 L179.7,496.1 L180.2,495.9 L180.7,495.7 L181.2,495.5 L181.7,495.3
 L182.1,495.1 L182.6,494.9 L183.1,494.8 L183.6,494.5 L184.1,494.4 L184.6,494.3 L185.1,494.2 L185.5,494.1
 L186.0,494.1 L186.5,494.0 L187.0,494.0 L187.5,493.9 L188.0,493.9 L188.5,493.9 L188.9,494.0 L189.4,

In [ ]:
var gp = PlotTableEnrgy[7,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.145 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M70.5,50.5 L71.0,51.5 L71.5,52.7 L72.0,54.0 L72.4,55.5 L72.9,57.1
 L73.4,59.0 L73.9,61.1 L74.4,63.6 L74.9,66.4 L75.4,69.5 L75.8,72.9 L76.3,76.6 L76.8,80.7
 L77.3,85.0 L77.8,89.8 L78.3,95.0 L78.8,100.4 L79.2,106.2 L79.7,112.4 L80.2,118.8 L80.7,125.5
 L81.2,132.6 L81.7,139.9 L82.2,147.4 L82.6,155.3 L83.1,163.3 L83.6,171.3 L84.1,179.8 L84.6,188.4
 L85.1,197.1 L85.5,205.9 L86.0,214.8 L86.5,223.7 L87.0,232.7 L87.5,241.6 L88.0,250.5 L88.5,259.4
 L88.9,268.1 L89.4,276.8 L89.9,285.3 L90.4,293.7 L90.9,301.9 L91.4,309.9 L91.9,317.7 L92.3,325.2
 L92.8,332.5 L93.3,339.6 L93.8,346.4 L94.3,352.9 L94.8,359.1 L95.3,365.0 L95.7,370.4 L96.2,375.8
 L96.7,380.9 L97.2,385.4 L97.7,389.9 L98.2,394.0 L98.7,397.8 L99.1,401.3 L99.6,404.5 L100.1,407.5
 L100.6,410.1 L101.1,412.5 L101.6,414.7 L102.1,416.6 L102.5,417.9 L103.0,419.6 L103.5,421.0 L104.0,422.1
 L104.5,423.2 L105.0,424.0 L105.5,424.8 L105.9,425.4 L106.4,425.9 L106.9,426.4 L107.4,426.8 L107.9,427.1
 L108.4,427.4 L108.8,427.7 L109.3,428.0 L109.8,428.3 L110.3,428.6 L110.8,428.9 L111.3,429.3 L111.8,429.7
 L112.2,430.2 L112.7,430.8 L113.2,431.4 L113.7,432.0 L114.2,432.8 L114.7,433.6 L115.2,434.6 L115.6,435.6
 L116.1,436.7 L116.6,437.8 L117.1,439.1 L117.6,440.5 L118.1,441.9 L118.6,443.4 L119.0,445.0 L119.5,446.7
 L120.0,448.2 L120.5,450.1 L121.0,452.0 L121.5,453.9 L122.0,455.8 L122.4,457.8 L122.9,459.9 L123.4,461.9
 L123.9,464.0 L124.4,466.0 L124.9,468.1 L125.4,470.2 L125.8,472.3 L126.3,474.4 L126.8,476.4 L127.3,478.4
 L127.8,480.4 L128.3,482.2 L128.8,484.2 L129.2,486.0 L129.7,487.8 L130.2,489.6 L130.7,491.3 L131.2,493.0
 L131.7,494.6 L132.1,496.1 L132.6,497.6 L133.1,499.0 L133.6,500.3 L134.1,501.6 L134.6,502.8 L135.1,503.9
 L135.5,505.0 L136.0,506.0 L136.5,506.9 L137.0,507.7 L137.5,508.5 L138.0,509.3 L138.5,509.9 L138.9,510.6
 L139.4,511.1 L139.9,511.6 L140.4,512.1 L140.9,512.5 L141.4,512.8 L141.9,513.2 L142.3,513.4 L142.8,513.7
 L143.3,513.9 L143.8,514.1 L144.3,514.2 L144.8,514.4 L145.3,514.5 L145.7,514.6 L146.2,514.7 L146.7,514.7
 L147.2,514.8 L147.7,514.9 L148.2,514.9 L148.7,515.0 L149.1,515.1 L149.6,515.1 L150.1,515.2 L150.6,515.3
 L151.1,515.4 L151.6,515.5 L152.1,515.6 L152.5,515.8 L153.0,515.9 L153.5,516.1 L154.0,516.3 L154.5,516.5
 L155.0,516.7 L155.5,516.9 L155.9,517.2 L156.4,517.5 L156.9,517.8 L157.4,518.1 L157.9,518.4 L158.4,518.8
 L158.8,519.1 L159.3,519.5 L159.8,519.9 L160.3,520.3 L160.8,520.7 L161.3,521.2 L161.8,521.6 L162.2,522.0
 L162.7,522.5 L163.2,522.9 L163.7,523.4 L164.2,523.9 L164.7,524.3 L165.2,524.8 L165.6,525.3 L166.1,525.7
 L166.6,526.2 L167.1,526.6 L167.6,527.0 L168.1,527.5 L168.6,527.9 L169.0,528.3 L169.5,528.7 L170.0,529.1
 L170.5,529.5 L171.0,529.8 L171.5,530.2 L172.0,530.5 L172.4,530.8 L172.9,531.2 L173.4,531.4 L173.9,531.7
 L174.4,532.0 L174.9,532.2 L175.4,532.5 L175.8,532.7 L176.3,532.9 L176.8,533.0 L177.3,533.2 L177.8,533.4
 L178.3,533.5 L178.8,533.6 L179.2,533.8 L179.7,533.9 L180.2,534.0 L180.7,534.0 L181.2,534.1 L181.7,534.2
 L182.1,534.2 L182.6,534.3 L183.1,534.3 L183.6,534.2 L184.1,534.3 L184.6,534.4 L185.1,534.5 L185.5,534.5
 L186.0,534.5 L186.5,534.6 L187.0,534.6 L187.5,534.6 L188.0,534.6 L188.5,534.6 L188.9,534.7 L189.4,534.7
 L189.9,534.7 L190.4,534.8 L190.9

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[7,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m4/" + caseS[7] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}

## Case 9 (m4_Oh056_eta005)

In [ ]:
var gp = PlotTableEnrgy[8,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.01 
 
 
 
 
 0 
 
 
 
 
 0.01 
 
 
 
 
 0.02 
 
 
 
 
 0.03 
 
 
 
 
 0.04 
 
 
 
 
 0.05 
 
 
 
 
 0.06 
 
 
 
 
 0.07 
 
 
 
 
 0.08 
 
 
 
 
 0.09 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M62.2,469.9 L62.7,469.9 L63.2,470.0 L63.7,470.1 L64.2,470.3 L64.7,470.6
 L65.1,471.0 L65.6,471.3 L66.1,471.8 L66.6,472.3 L67.1,472.8 L67.6,473.4 L68.1,474.0 L68.6,474.7
 L69.1,475.4 L69.6,476.1 L70.1,476.8 L70.6,477.6 L71.0,478.4 L71.5,479.1 L72.0,479.9 L72.5,480.7
 L73.0,481.5 L73.5,482.3 L74.0,483.2 L74.5,484.0 L75.0,484.8 L75.5,485.5 L76.0,486.3 L76.4,487.1
 L76.9,487.8 L77.4,488.6 L77.9,489.3 L78.4,490.0 L78.9,490.7 L79.4,491.4 L79.9,492.1 L80.4,492.7
 L80.9,493.3 L81.4,493.9 L81.9,494.5 L82.3,495.1 L82.8,495.7 L83.3,496.2 L83.8,496.7 L84.3,497.2
 L84.8,497.7 L85.3,498.1 L85.8,498.6 L86.3,499.0 L86.8,499.4 L87.3,499.8 L87.8,500.2 L88.2,500.5
 L88.7,500.9 L89.2,501.2 L89.7,501.5 L90.2,501.8 L90.7,502.1 L91.2,502.4 L91.7,502.6 L92.2,502.9
 L92.7,503.1 L93.2,503.3 L93.6,503.6 L94.1,503.8 L94.6,504.0 L95.1,504.1 L95.6,504.3 L96.1,504.5
 L96.6,504.7 L97.1,504.8 L97.6,505.0 L98.1,505.1 L98.6,505.2 L99.1,505.4 L99.5,505.5 L100.0,505.6
 L100.5,505.7 L101.0,505.8 L101.5,505.9 L102.0,506.0 L102.5,506.1 L103.0,506.2 L103.5,506.3 L104.0,506.3
 L104.5,506.4 L104.9,506.5 L105.4,506.6 L105.9,506.6 L106.4,506.7 L106.9,506.8 L107.4,506.8 L107.9,506.9
 L108.4,507.0 L108.9,507.0 L109.4,507.1 L109.9,507.1 L110.4,507.2 L110.8,507.2 L111.3,507.3 L111.8,507.3
 L112.3,507.4 L112.8,507.4 L113.3,507.4 L113.8,507.5 L114.3,507.5 L114.8,507.6 L115.3,507.6 L115.8,507.7
 L116.2,507.7 L116.7,507.8 L117.2,507.8 L117.7,507.8 L118.2,507.9 L118.7,507.9 L119.2,508.0 L119.7,508.0
 L120.2,508.0 L120.7,508.1 L121.2,508.1 L121.7,508.2 L122.1,508.2 L122.6,508.2 L123.1,508.3 L123.6,508.3
 L124.1,508.4 L124.6,508.4 L125.1,508.4 L125.6,508.5 L126.1,508.5 L126.6,508.5 L127.1,508.6 L127.6,508.6
 L128.0,508.7 L128.5,508.7 L129.0,508.7 L129.5,508.8 L130.0,508.8 L130.5,508.8 L131.0,508.9 L131.5,508.9
 L132.0,509.0 L132.5,509.0 L133.0,509.0 L133.4,509.1 L133.9,509.1 L134.4,509.1 L134.9,509.2 L135.4,509.2
 L135.9,509.2 L136.4,509.3 L136.9,509.3 L137.4,509.3 L137.9,509.4 L138.4,509.4 L138.9,509.4 L139.3,509.5
 L139.8,509.5 L140.3,509.5 L140.8,509.6 L141.3,509.6 L141.8,509.6 L142.3,509.7 L142.8,509.7 L143.3,509.7
 L143.8,509.8 L144.3,509.8 L144.7,509.8 L145.2,509.9 L145.7,509.9 L146.2,509.9 L146.7,509.9 L147.2,510.0
 L147.7,510.0 L148.2,510.0 L148.7,510.1 L149.2,510.1 L149.7,510.1 L150.2,510.1 L150.6,510.2 L151.1,510.2
 L151.6,510.2 L152.1,510.2 L152.6,510.3 L153.1,510.3 L153.6,510.3 L154.1,510.3 L154.6,510.4 L155.1,510.4
 L155.6,510.4 L156.0,510.4 L156.5,510.5 L157.0,510.5 L157.5,510.5 L158.0,510.5 L158.5,510.5 L159.0,510.6
 L159.5,510.6 L160.0,510.6 L160.5,510.6 L161.0,510.6 L161.5,510.7 L161.9,510.7 L162.4,510.7 L162.9,510.7
 L163.4,510.7 L163.9,510.8 L164.4,510.8 L164.9,510.8 L165.4,510.8 L165.9,510.8 L166.4,510.8 L166.9,510.9
 L167.4,510.9 L167.8,510.9 L168.3,510.9 L168.8,510.9 L169.3,510.9 L169.8,510.9 L170.3,511.0 L170.8,511.0
 L171.3,511.0 L171.8,511.0 L172.3,511.0 L172.8,511.0 L173.2,511.0 L173.7,511.1 L174.2,511.1 L174.7,511.1
 L175.2,511.1 L175.7,511.1 L176.2,511.1 L176.7,511.1 L177.2,511.1 L177.7,511.1 L178.2,511.1 L178.7,511.2
 L179.1,511.2 L179.6,511.2 L180.1,511.2 L180.6,511.2 L181.1,511.

In [ ]:
var gp = PlotTableEnrgy[8,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.15 
 
 
 
 
 3.16 
 
 
 
 
 3.17 
 
 
 
 
 3.18 
 
 
 
 
 3.19 
 
 
 
 
 3.2 
 
 
 
 
 3.21 
 
 
 
 
 3.22 
 
 
 
 
 3.23 
 
 
 
 
 3.24 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M62.2,79.2 L62.7,145.1 L63.2,201.8 L63.7,250.0 L64.2,290.5 L64.7,324.3
 L65.1,352.7 L65.6,376.5 L66.1,396.6 L66.6,413.5 L67.1,427.8 L67.6,440.0 L68.1,450.4 L68.6,459.2
 L69.1,466.8 L69.6,473.4 L70.1,479.1 L70.6,484.0 L71.0,488.4 L71.5,492.2 L72.0,495.6 L72.5,498.6
 L73.0,501.4 L73.5,503.9 L74.0,506.2 L74.5,508.3 L75.0,510.2 L75.5,512.0 L76.0,513.7 L76.4,515.3
 L76.9,516.9 L77.4,518.3 L77.9,519.7 L78.4,521.0 L78.9,522.3 L79.4,523.5 L79.9,524.7 L80.4,525.8
 L80.9,526.9 L81.4,527.9 L81.9,528.9 L82.3,529.9 L82.8,530.9 L83.3,531.8 L83.8,532.7 L84.3,533.5
 L84.8,534.3 L85.3,535.1 L85.8,535.9 L86.3,536.6 L86.8,537.3 L87.3,538.0 L87.8,538.7 L88.2,539.3
 L88.7,539.9 L89.2,540.5 L89.7,541.1 L90.2,541.6 L90.7,542.1 L91.2,542.6 L91.7,543.1 L92.2,543.5
 L92.7,544.0 L93.2,544.4 L93.6,544.8 L94.1,545.1 L94.6,545.5 L95.1,545.9 L95.6,546.2 L96.1,546.5
 L96.6,546.8 L97.1,547.1 L97.6,547.3 L98.1,547.6 L98.6,547.8 L99.1,548.1 L99.5,548.3 L100.0,548.5
 L100.5,548.7 L101.0,548.9 L101.5,549.1 L102.0,549.2 L102.5,549.4 L103.0,549.6 L103.5,549.7 L104.0,549.8
 L104.5,550.0 L104.9,550.1 L105.4,550.2 L105.9,550.3 L106.4,550.4 L106.9,550.5 L107.4,550.6 L107.9,550.7
 L108.4,550.8 L108.9,550.9 L109.4,551.0 L109.9,551.0 L110.4,551.1 L110.8,551.2 L111.3,551.2 L111.8,551.3
 L112.3,551.4 L112.8,551.4 L113.3,551.4 L113.8,551.5 L114.3,551.5 L114.8,551.6 L115.3,551.6 L115.8,551.7
 L116.2,551.7 L116.7,551.8 L117.2,551.8 L117.7,551.9 L118.2,551.9 L118.7,551.9 L119.2,552.0 L119.7,552.0
 L120.2,552.0 L120.7,552.1 L121.2,552.1 L121.7,552.1 L122.1,552.2 L122.6,552.2 L123.1,552.2 L123.6,552.3
 L124.1,552.3 L124.6,552.3 L125.1,552.3 L125.6,552.4 L126.1,552.4 L126.6,552.4 L127.1,552.5 L127.6,552.5
 L128.0,552.5 L128.5,552.5 L129.0,552.6 L129.5,552.6 L130.0,552.6 L130.5,552.7 L131.0,552.7 L131.5,552.7
 L132.0,552.7 L132.5,552.8 L133.0,552.8 L133.4,552.8 L133.9,552.9 L134.4,552.9 L134.9,552.9 L135.4,552.9
 L135.9,553.0 L136.4,553.0 L136.9,553.0 L137.4,553.1 L137.9,553.1 L138.4,553.1 L138.9,553.2 L139.3,553.2
 L139.8,553.2 L140.3,553.2 L140.8,553.3 L141.3,553.3 L141.8,553.3 L142.3,553.4 L142.8,553.4 L143.3,553.4
 L143.8,553.4 L144.3,553.5 L144.7,553.5 L145.2,553.5 L145.7,553.6 L146.2,553.6 L146.7,553.6 L147.2,553.6
 L147.7,553.7 L148.2,553.7 L148.7,553.7 L149.2,553.8 L149.7,553.8 L150.2,553.8 L150.6,553.9 L151.1,553.9
 L151.6,553.9 L152.1,553.9 L152.6,554.0 L153.1,554.0 L153.6,554.0 L154.1,554.0 L154.6,554.1 L155.1,554.1
 L155.6,554.1 L156.0,554.2 L156.5,554.2 L157.0,554.2 L157.5,554.2 L158.0,554.3 L158.5,554.3 L159.0,554.3
 L159.5,554.3 L160.0,554.4 L160.5,554.4 L161.0,554.4 L161.5,554.4 L161.9,554.5 L162.4,554.5 L162.9,554.5
 L163.4,554.5 L163.9,554.6 L164.4,554.6 L164.9,554.6 L165.4,554.6 L165.9,554.7 L166.4,554.7 L166.9,554.7
 L167.4,554.7 L167.8,554.7 L168.3,554.8 L168.8,554.8 L169.3,554.8 L169.8,554.8 L170.3,554.8 L170.8,554.9
 L171.3,554.9 L171.8,554.9 L172.3,554.9 L172.8,554.9 L173.2,555.0 L173.7,555.0 L174.2,555.0 L174.7,555.0
 L175.2,555.0 L175.7,555.0 L176.2,555.1 L176.7,555.1 L177.2,555.1 L177.7,555.1 L178.2,555.1 L178.7,555.1
 L179.1,555.2 L179.6,555.2 L180.1,555.2 L180.6,555.2 L181.1,555.2 L1

In [ ]:
for(int i = 0; i < 2; i++) {
    foreach(var grp in PlotTableEnrgy[8,i].dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/Energies/m4/" + caseS[8] + " " + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
}